# WILDFIRESAI - DATA PIPELINE (Base Notebook)

# ---------------------------------------------------------------------
# ==== Cell 1 — Setup  ====
 Purpose:
   Install and validate core dependencies required by WildfiresAI.
   Ensure modules are importable without kernel restart.
# ---------------------------------------------------------------------


In [1]:
# ==== Cell 1 — Setup (portable dependency install) ====
# Installs missing dependencies and validates imports across environments.

from typing import Dict
import importlib, importlib.util, subprocess, sys, site

REQUIRED: Dict[str, str] = {
    "pystac-client": "pystac_client",
    "planetary-computer": "planetary_computer",
    "mp-api": "mp_api",
    "pymatgen": "pymatgen",
    "pyarrow": "pyarrow",
    "tqdm": "tqdm",
    "structlog": "structlog",
}

def _is_importable(mod): return importlib.util.find_spec(mod) is not None
def _install(pkg): subprocess.run([sys.executable, "-m", "pip", "install", "--quiet", pkg], check=True)

# Append user site (portable fix for JupyterHub)
try:
    user_site = site.getusersitepackages()
    if user_site not in sys.path:
        sys.path.append(user_site)
except Exception:
    pass

for pkg, mod in REQUIRED.items():
    if not _is_importable(mod): _install(pkg)

importlib.invalidate_caches()
status = {}
for pkg, mod in REQUIRED.items():
    try:
        m = importlib.import_module(mod)
        status[mod] = getattr(m, "__version__", "ok")
    except Exception as e:
        status[mod] = f"import failed ({e})"

print("[Installed modules status]")
for m, v in status.items():
    print(f" - {m}: {v}")
print("\nSetup complete.")


[Installed modules status]
 - pystac_client: 0.9.0
 - planetary_computer: 1.0.0
 - mp_api: ok
 - pymatgen: ok
 - pyarrow: 21.0.0
 - tqdm: 4.67.1
 - structlog: 25.4.0

Setup complete.


# ---------------------------------------------------------------------
# ==== Cell 2 — Scientific / Infra Stack & Version Audit ====
 Purpose:
   Establish project-wide environment paths and verify
   availability & versions of the core scientific stack.
# ---------------------------------------------------------------------


In [2]:
# ==== Cell 2 — Scientific / Infra Stack & Version Audit ====
# Purpose:
# - Define project paths used across AG² components.
# - Produce a concise environment + versions audit (core scientific + geo stack).
# - Verify write permissions on critical directories.
#
# Notes:
# - Comments are concise and technical (MIT Lab style).
# - No duplicated installations/import policies from Cell 1.

from __future__ import annotations
import os, platform, importlib, tempfile
from pathlib import Path
import importlib.util
import pandas as pd

# ----------------------------- Project Paths -----------------------------
PROJECT_ROOT  = Path.cwd()
DATA_DIR      = PROJECT_ROOT / "data"
RAW_DIR       = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR   = PROJECT_ROOT / "reports"
LOGS_DIR      = REPORTS_DIR / "logs"

for d in (RAW_DIR, PROCESSED_DIR, REPORTS_DIR, LOGS_DIR):
    d.mkdir(parents=True, exist_ok=True)

# Optional: write-permission probe (detects NFS/readonly issues early)
def _can_write(path: Path) -> bool:
    try:
        with tempfile.TemporaryFile(dir=path):
            pass
        return True
    except Exception:
        return False

perm = {
    "data/raw": _can_write(RAW_DIR),
    "data/processed": _can_write(PROCESSED_DIR),
    "reports": _can_write(REPORTS_DIR),
    "reports/logs": _can_write(LOGS_DIR),
}

# ----------------------------- ENV Summary -------------------------------
print(f"[ENV] Python {platform.python_version()} | Platform: {platform.system()} {platform.release()}")
print(f"[ENV] Root directory: {PROJECT_ROOT}")
print("[ENV] Folders (exists / writable):")
print(f"   - data/raw:        {RAW_DIR.exists()} / {perm['data/raw']}")
print(f"   - data/processed:  {PROCESSED_DIR.exists()} / {perm['data/processed']}")
print(f"   - reports/:        {REPORTS_DIR.exists()} / {perm['reports']}")
print(f"   - reports/logs/:   {LOGS_DIR.exists()} / {perm['reports/logs']}")

# ------------------------- Core Stack Versions ---------------------------
core_packages = [
    "pandas", "numpy", "requests", "geopandas", "rasterio",
    "shapely", "matplotlib", "tqdm", "sklearn", "torch"
]

versions = {}
for pkg in core_packages:
    try:
        mod = importlib.import_module(pkg)
        versions[pkg] = getattr(mod, "__version__", "ok")
    except Exception as e:
        versions[pkg] = f"not installed ({e.__class__.__name__})"

df_versions = pd.DataFrame.from_dict(versions, orient="index", columns=["version"])
display(df_versions.T.style.set_caption("Core Scientific Stack Versions"))

# ---------------------- Geo Stack & Backend Diagnostics -------------------
diag = {}

# Geo: PROJ/GDAL, Fiona, PyProj availability (common failure modes)
def _is_importable(name: str) -> bool:
    return importlib.util.find_spec(name) is not None

if _is_importable("geopandas"):
    try:
        import geopandas as gpd
        diag["geopandas_crs_ok"] = True
    except Exception as e:
        diag["geopandas_crs_ok"] = f"error ({e.__class__.__name__})"

if _is_importable("pyproj"):
    try:
        import pyproj
        from pyproj.datadir import get_data_dir
        diag["pyproj_version"] = getattr(pyproj, "__version__", "ok")
        diag["proj_data_dir"] = get_data_dir()
    except Exception as e:
        diag["pyproj_version"] = f"error ({e.__class__.__name__})"

if _is_importable("fiona"):
    try:
        import fiona
        diag["fiona_version"] = getattr(fiona, "__version__", "ok")
        diag["fiona_gdal"] = fiona.env.get_gdal_release_name()
    except Exception as e:
        diag["fiona_version"] = f"error ({e.__class__.__name__})"

if _is_importable("rasterio"):
    try:
        import rasterio
        diag["rasterio_gdal"] = getattr(rasterio, "__gdal_version__", "unknown")
        diag["rasterio_proj"] = getattr(rasterio, "__proj_version__", "unknown")
    except Exception as e:
        diag["rasterio_gdal"] = f"error ({e.__class__.__name__})"

# Torch: CUDA/MPS/CPU backend report (Macs often use MPS)
if _is_importable("torch"):
    try:
        import torch
        backend = "cpu"
        if hasattr(torch, "cuda") and torch.cuda.is_available():
            backend = f"cuda:{torch.cuda.get_device_name(0)}"
        elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
            backend = "mps"
        diag["torch_backend"] = backend
        diag["torch_dtype_default"] = str(torch.get_default_dtype()) if hasattr(torch, "get_default_dtype") else "float32"
    except Exception as e:
        diag["torch_backend"] = f"error ({e.__class__.__name__})"

# Consolidated diagnostics (readable print to avoid DataFrame overhead here)
print("\n[Diagnostics]")
for k, v in diag.items():
    print(f" - {k}: {v}")

print("\nEnvironment and version audit completed successfully.")


[ENV] Python 3.13.5 | Platform: Darwin 24.6.0
[ENV] Root directory: /Users/evareysanchez/WildfiresAI
[ENV] Folders (exists / writable):
   - data/raw:        True / True
   - data/processed:  True / True
   - reports/:        True / True
   - reports/logs/:   True / True


,pandas,numpy,requests,geopandas,rasterio,shapely,matplotlib,tqdm,sklearn,torch
version,2.3.2,2.3.3,2.32.4,1.1.1,1.4.3,2.1.2,3.10.6,4.67.1,1.7.2,2.8.0



[Diagnostics]
 - geopandas_crs_ok: True
 - pyproj_version: 3.7.2
 - proj_data_dir: /opt/miniconda3/lib/python3.13/site-packages/pyproj/proj_dir/share/proj
 - fiona_version: 1.10.1
 - fiona_gdal: 3.9.2
 - rasterio_gdal: 3.9.3
 - rasterio_proj: 9.4.1
 - torch_backend: mps
 - torch_dtype_default: torch.float32

Environment and version audit completed successfully.


# ---------------------------------------------------------------------
# ==== Cell 2.1 — Keys & Connectivity Audit ====

Purpose:
- Verify API keys and data endpoints availability.
- Perform lightweight connectivity and authorization checks.
- Produce a structured JSON report saved to /reports/connectivity_report.json
- Includes NASA FIRMS, Materials Project, OpenAI, and other key integrations.
# ---------------------------------------------------------------------



In [3]:
# ==== Cell 2.1 — Keys & Connectivity Audit ====

from __future__ import annotations
import os, json, time
from pathlib import Path
from datetime import date, timedelta, datetime, timezone
from urllib.parse import urlparse
from typing import Dict, Any
import importlib.util
import concurrent.futures as cf

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ------------------------------ Helpers ---------------------------------
def _is_importable(name: str) -> bool:
    return importlib.util.find_spec(name) is not None

def mask(key: str, keep: int = 6) -> str:
    if not key: return "None"
    return key[:keep] + "…" if len(key) > keep else "***"

def ok(status: bool, reason: str | None = None, extra: dict | None = None) -> dict:
    out = {"ok": bool(status)}
    if reason: out["reason"] = reason
    if extra: out.update(extra)
    return out

def _force_effis_base(url: str | None) -> str:
    default = "https://maps.effis.emergency.copernicus.eu/effis"
    if not url: return default
    host = urlparse(url).netloc.lower()
    if "forest-fire.jrc.ec.europa.eu" in host or "jrc.ec.europa.eu" in host:
        return default
    return url

def _mk_session(total=2, backoff=0.5, status=(429, 502, 503, 504)) -> requests.Session:
    s = requests.Session()
    r = Retry(total=total, backoff_factor=backoff, status_forcelist=list(status), raise_on_status=False)
    s.mount("https://", HTTPAdapter(max_retries=r))
    s.mount("http://", HTTPAdapter(max_retries=r))
    return s

def _timed_request(method: str, url: str, *, timeout=(4, 8), session: requests.Session | None = None, **kwargs) -> Dict[str, Any]:
    s = session or _mk_session()
    t0 = time.perf_counter()
    try:
        resp = s.request(method, url, timeout=timeout, **kwargs)
        dt = time.perf_counter() - t0
        return {
            "ok": resp.ok,
            "status_code": getattr(resp, "status_code", None),
            "elapsed_s": round(dt, 3),
            "content_type": resp.headers.get("Content-Type", ""),
            "degraded": round(dt, 3) > 15
        }
    except Exception as e:
        dt = time.perf_counter() - t0
        return {"ok": False, "reason": f"{e.__class__.__name__}: {e}", "elapsed_s": round(dt, 3)}

# --------------------------- Environment ---------------------------------
env = {
    "WF_REGION": os.getenv("WF_REGION", "ES"),
    "WF_DATE_FROM": os.getenv("WF_DATE_FROM"),
    "WF_DATE_TO": os.getenv("WF_DATE_TO"),
    "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY", ""),
    "MP_API_KEY": os.getenv("MP_API_KEY", ""),
    "OPENTOPO_API_KEY": os.getenv("OPENTOPO_API_KEY", ""),
    "NASA_FIRMS_TOKEN": os.getenv("NASA_FIRMS_TOKEN", ""),
    "EFFIS_WFS_URL": _force_effis_base(os.getenv("EFFIS_WFS_URL", "https://maps.effis.emergency.copernicus.eu/effis")),
    "EFFIS_WMS_URL": _force_effis_base(os.getenv("EFFIS_WMS_URL", "https://maps.effis.emergency.copernicus.eu/effis")),
    "EFFIS_WCS_URL": _force_effis_base(os.getenv("EFFIS_WCS_URL", "https://maps.effis.emergency.copernicus.eu/effis")),
    "EFFIS_TYPENAME": os.getenv("EFFIS_TYPENAME", "ms:modis.ba.poly"),
    "NIFC_FS_URL": os.getenv("NIFC_FS_URL",
        "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/"
        "WFIGS_Interagency_Perimeters_Current/FeatureServer/0"),
    "EARTHDATA_TOKEN": os.getenv("EARTHDATA_TOKEN", ""),
}
print(f"[ENV] Region/Window: {env['WF_REGION']} {env['WF_DATE_FROM']} → {env['WF_DATE_TO']}")
print("[ENV] Keys (masked):",
      "OPENAI=", mask(env["OPENAI_API_KEY"]),
      "MP=", mask(env["MP_API_KEY"]),
      "OPENTOPO=", mask(env["OPENTOPO_API_KEY"]),
      "NASA_FIRMS_TOKEN=", mask(env["NASA_FIRMS_TOKEN"]),
      "EARTHDATA_TOKEN=", mask(env["EARTHDATA_TOKEN"]))

# ----------------------------- Tasks -------------------------------------
def check_openai() -> dict:
    key = env["OPENAI_API_KEY"]
    if not key:
        return ok(False, "OPENAI_API_KEY missing")
    s = _mk_session()
    r = _timed_request(
        "GET", "https://api.openai.com/v1/models",
        headers={"Authorization": f"Bearer {key}"},
        timeout=(4, 8), session=s
    )
    return ok(r["ok"], None if r["ok"] else r.get("reason"), r)

def check_mp() -> dict:
    if not env["MP_API_KEY"]:
        return ok(False, "MP_API_KEY missing")
    try:
        from mp_api.client import MPRester
        t0 = time.perf_counter()
        with MPRester(env["MP_API_KEY"]) as mpr:
            docs = mpr.materials.summary.search(fields=["material_id"], chunk_size=1, num_chunks=1)
        dt = round(time.perf_counter() - t0, 3)
        return ok(True, extra={"sample_docs": len(docs), "elapsed_s": dt})
    except Exception as e:
        return ok(False, f"mp-api error: {e.__class__.__name__}: {e}")

def check_opentopo() -> dict:
    if not env["OPENTOPO_API_KEY"]:
        return ok(False, "OPENTOPO_API_KEY missing")
    s = _mk_session()
    r = _timed_request(
        "HEAD", "https://portal.opentopography.org/API/globaldem",
        params=dict(
            demtype="SRTMGL3", south=40.0, north=40.1, west=-3.8, east=-3.7,
            outputFormat="GTiff", API_Key=env["OPENTOPO_API_KEY"]
        ),
        timeout=(10, 25), session=s  # increased for reliability
    )
    return ok(r["ok"], None if r["ok"] else r.get("reason"), r)

def check_open_meteo() -> dict:
    s = _mk_session()
    today = date.today()
    params = dict(latitude=40.0, longitude=-3.7,
                  start_date=(today - timedelta(days=2)).isoformat(),
                  end_date=(today - timedelta(days=1)).isoformat(),
                  hourly="temperature_2m", timezone="UTC")
    r = _timed_request("GET", "https://archive-api.open-meteo.com/v1/archive",
                       params=params, timeout=(4, 8), session=s)
    if not r["ok"]:
        return ok(False, r.get("reason"), r)
    try:
        js = requests.get("https://archive-api.open-meteo.com/v1/archive", params=params, timeout=(4, 8)).json()
        valid = "hourly" in js and "time" in js["hourly"]
        return ok(valid, None if valid else "unexpected JSON", {**r, "json_keys": list(js.keys())})
    except Exception as e:
        return ok(False, f"JSON parse: {e.__class__.__name__}: {e}", r)

def check_firms() -> dict:
    firms_env = {k: v for k, v in os.environ.items() if k.startswith("FIRMS_") and v.startswith("https")}
    token = env["NASA_FIRMS_TOKEN"]
    if not firms_env:
        return ok(False, "No FIRMS_* URLs found")
    s = _mk_session()
    valid = 0
    for name, url in firms_env.items():
        headers = {"Authorization": f"Bearer {token}"} if token else {}
        r = _timed_request("HEAD", url, headers=headers, timeout=(4, 8), session=s)
        if r["ok"]: valid += 1
    return ok(valid > 0, extra={"feeds_detected": len(firms_env), "validated": valid})

def check_effis() -> dict:
    """Resilient EFFIS capability check with extended timeouts."""
    base_wfs = _force_effis_base(env["EFFIS_WFS_URL"]).rstrip("/")
    base_wms = _force_effis_base(env["EFFIS_WMS_URL"]).rstrip("/")
    base_wcs = _force_effis_base(env["EFFIS_WCS_URL"]).rstrip("/")
    s = _mk_session(total=2, backoff=0.6)
    headers_xml = {"Accept": "application/xml"}

    # 1) HEAD ping
    head_wfs = _timed_request("HEAD", base_wfs, timeout=(6, 10), session=s)
    head_wms = _timed_request("HEAD", base_wms, timeout=(6, 10), session=s)
    head_wcs = _timed_request("HEAD", base_wcs, timeout=(6, 10), session=s)

    # 2) Light GetCapabilities
    def _cap(url, service):
        return _timed_request(
            "GET", url,
            params={"service": service, "request": "GetCapabilities", "version": "1.1.1"},
            headers={**headers_xml, "Range": "bytes=0-2047"},
            timeout=(10, 20), session=s
        )
    wfs_cap = _cap(base_wfs, "WFS") if head_wfs["ok"] else {"ok": False}
    wms_cap = _cap(base_wms, "WMS") if head_wms["ok"] else {"ok": False}
    wcs_cap = _cap(base_wcs, "WCS") if head_wcs["ok"] else {"ok": False}

    # 3) Tiny GetMap ping
    wms_ping = {"ok": False}
    if wms_cap["ok"]:
        yday = (date.today() - timedelta(days=1)).isoformat()
        params = {
            "SERVICE": "WMS", "REQUEST": "GetMap", "VERSION": "1.1.1",
            "LAYERS": "ecmwf007.fwi", "STYLES": "",
            "SRS": "EPSG:4326", "BBOX": "-18,27,42,72",
            "WIDTH": 1, "HEIGHT": 1, "FORMAT": "image/png",
            "TRANSPARENT": "true", "TIME": yday
        }
        wms_ping = _timed_request("GET", base_wms, params=params, timeout=(10, 20), session=s)

    effis = {
        "wfs_head": head_wfs, "wms_head": head_wms, "wcs_head": head_wcs,
        "wfs_cap": wfs_cap, "wms_cap": wms_cap, "wcs_cap": wcs_cap,
        "wms_getmap_ping": wms_ping
    }
    ok_all = all(v.get("ok", False) for v in effis.values())
    effis["degraded"] = any(v.get("degraded") for v in effis.values())
    return {"ok": ok_all, **effis}

def check_nifc() -> dict:
    base = env["NIFC_FS_URL"].rstrip("/")
    r = _timed_request("HEAD", f"{base}?f=json", timeout=(4, 8))
    return ok(r["ok"], None if r["ok"] else r.get("reason"), r)

def check_earthdata() -> dict:
    has = bool(env.get("EARTHDATA_TOKEN"))
    return ok(has, None if has else "EARTHDATA_TOKEN missing")

# --------------------------- Concurrent run ------------------------------
tasks = {
    "openai": check_openai,
    "materials_project": check_mp,
    "opentopo": check_opentopo,
    "open_meteo": check_open_meteo,
    "firms": check_firms,
    "effis": check_effis,
    "nifc": check_nifc,
    "earthdata": check_earthdata,
}

report: Dict[str, dict] = {}
with cf.ThreadPoolExecutor(max_workers=min(8, len(tasks))) as ex:
    fut_map = {ex.submit(fn): name for name, fn in tasks.items()}
    for fut in cf.as_completed(fut_map):
        name = fut_map[fut]
        try:
            report[name] = fut.result()
        except Exception as e:
            report[name] = ok(False, f"task error: {e.__class__.__name__}: {e}")

# ----------------------------- Persist -----------------------------------
out = {
    "generated_at": datetime.now(timezone.utc).isoformat(),
    "region": env["WF_REGION"],
    "window": {"from": env["WF_DATE_FROM"], "to": env["WF_DATE_TO"]},
    "results": dict(sorted(report.items(), key=lambda kv: kv[0])),
}
out_path = Path("reports") / "connectivity_report.json"
out_path.write_text(json.dumps(out, indent=2, ensure_ascii=False), encoding="utf-8")
print(f"\nConnectivity report written to {out_path}")
print(json.dumps(out, indent=2, ensure_ascii=False))


[ENV] Region/Window: ES None → None
[ENV] Keys (masked): OPENAI= sk-pro… MP= U8Wg4j… OPENTOPO= 9d7124… NASA_FIRMS_TOKEN= eyJ0eX… EARTHDATA_TOKEN= eyJ0eX…


Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]


Connectivity report written to reports/connectivity_report.json
{
  "generated_at": "2025-11-04T18:29:05.322936+00:00",
  "region": "ES",
  "window": {
    "from": null,
    "to": null
  },
  "results": {
    "earthdata": {
      "ok": true
    },
    "effis": {
      "ok": false,
      "wfs_head": {
        "ok": true,
        "status_code": 200,
        "elapsed_s": 0.62,
        "content_type": "text/html",
        "degraded": false
      },
      "wms_head": {
        "ok": true,
        "status_code": 200,
        "elapsed_s": 21.627,
        "content_type": "text/html",
        "degraded": true
      },
      "wcs_head": {
        "ok": true,
        "status_code": 200,
        "elapsed_s": 21.964,
        "content_type": "text/html",
        "degraded": true
      },
      "wfs_cap": {
        "ok": false,
        "reason": "ConnectionError: HTTPSConnectionPool(host='maps.effis.emergency.copernicus.eu', port=443): Max retries exceeded with url: /effis?service=WFS&request=GetCap

## Cell 3 – Project Header & Global Configuration
Defines project paths, environment variables, logging, and small I/O helpers.  
Ensures reproducibility, consistent data handling, and clean outputs across the pipeline.

In [4]:
# ==== WildfiresAI — Cell 3: Project Header & Global Configuration (Lab-grade) ====

from __future__ import annotations
from typing import Optional
from datetime import datetime, date, timedelta, timezone
import os, platform
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
import structlog
from tqdm import tqdm

# ----------------------------- Directories -------------------------------
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR = PROJECT_ROOT / "reports"
CONFIG_DIR = PROJECT_ROOT / "configs"

for p in (DATA_DIR, RAW_DIR, PROCESSED_DIR, REPORTS_DIR, CONFIG_DIR):
    p.mkdir(parents=True, exist_ok=True)

# ------------------------------ Environment ------------------------------
load_dotenv()  # Load from .env or system
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
FIRMS_TOKEN    = os.getenv("FIRMS_TOKEN", "")
FIRMS_MAP_KEY  = os.getenv("FIRMS_MAP_KEY", "")
WF_REGION      = os.getenv("WF_REGION", "GLOBAL").upper()
FGPL_MODE      = os.getenv("FGPL_MODE", "GLOBAL").upper()

VALID_REGIONS = {"GLOBAL", "ES"}
if WF_REGION not in VALID_REGIONS:
    WF_REGION = "GLOBAL"

# --------------------------- Spatial Configuration ----------------------
SPAIN_BBOX = (-9.5, 35.0, 3.5, 43.9)
ACTIVE_BBOX = None if WF_REGION == "GLOBAL" else SPAIN_BBOX

# ----------------------------- Structured Log ----------------------------
structlog.configure(
    processors=[
        structlog.processors.TimeStamper(fmt="iso", utc=True),
        structlog.processors.add_log_level,
        structlog.processors.JSONRenderer(),
    ],
    wrapper_class=structlog.make_filtering_bound_logger(20),  # INFO level
)
log = structlog.get_logger("wildfiresai").bind(region=WF_REGION, mode=FGPL_MODE)
log.info("Init config", root=str(PROJECT_ROOT), processed=str(PROCESSED_DIR))

# ------------------------------ I/O Helpers ------------------------------
def mask_key(key: str, n: int = 6) -> str:
    """Mask sensitive keys, keeping only first n chars."""
    return key[:n] + "…" if key else "None"

def save_df(df: pd.DataFrame, path: Path) -> None:
    """Save DataFrame to CSV (UTF-8, reproducible format)."""
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(path, index=False, float_format="%.6f", encoding="utf-8")
    log.info("Saved CSV", path=str(path), rows=len(df))

def save_parquet(df: pd.DataFrame, path: Path) -> None:
    """Save DataFrame to compressed Parquet."""
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_parquet(path, index=False, compression="snappy")
    log.info("Saved Parquet", path=str(path), rows=len(df))

def preview(df: pd.DataFrame, n: int = 6) -> pd.DataFrame:
    """Return the first n rows for consistent preview."""
    return df.head(n)

# --------------------------- Configuration Echo --------------------------
print("────────────────────────────────────────────────────────────")
print("WildfiresAI Global Configuration (Lab-grade)")
print("────────────────────────────────────────────────────────────")
print(f"Mode: {FGPL_MODE} | Region: {WF_REGION}")
if ACTIVE_BBOX:
    print(f"Bounding Box: {ACTIVE_BBOX}")
else:
    print("Bounding Box: GLOBAL coverage (no spatial restriction)")
print(f"Paths: raw={RAW_DIR.name}, processed={PROCESSED_DIR.name}, reports={REPORTS_DIR.name}")
print(f"Secrets: OpenAI={mask_key(OPENAI_API_KEY)}, FIRMS_TOKEN={mask_key(FIRMS_TOKEN)}, MAP_KEY={mask_key(FIRMS_MAP_KEY)}")
print(f"System: {platform.system()} {platform.release()} | Python {platform.python_version()}")
print(f"Timestamp (UTC): {datetime.now(timezone.utc).isoformat()}")
print("────────────────────────────────────────────────────────────")


{"region": "GLOBAL", "mode": "GLOBAL", "root": "/Users/evareysanchez/WildfiresAI", "processed": "/Users/evareysanchez/WildfiresAI/data/processed", "event": "Init config", "timestamp": "2025-11-04T18:29:24.987979Z", "level": "info"}
────────────────────────────────────────────────────────────
WildfiresAI Global Configuration (Lab-grade)
────────────────────────────────────────────────────────────
Mode: GLOBAL | Region: GLOBAL
Bounding Box: GLOBAL coverage (no spatial restriction)
Paths: raw=raw, processed=processed, reports=reports
Secrets: OpenAI=sk-pro…, FIRMS_TOKEN=eyJ0eX…, MAP_KEY=27f8d7…
System: Darwin 24.6.0 | Python 3.13.5
Timestamp (UTC): 2025-11-04T18:29:24.989553+00:00
────────────────────────────────────────────────────────────


# -------------------------------------------------------------------------
# ==== Cell 3.1: Smart Date Synchronization (Auto Window Detection, Global-Aware) ====

 Purpose:
 Automatically detect or initialize temporal window for global datasets.
 Ensures WF_DATE_FROM / WF_DATE_TO are synchronized across .env, system env,
 and global pipeline state (GLOBAL or REGIONAL modes).
# -------------------------------------------------------------------------

In [5]:
# ==== WildfiresAI — Cell 3.1: Smart Date Synchronization (UTC-safe, Global-Aware) ====


from __future__ import annotations
import os
from datetime import datetime, timedelta, timezone
from pathlib import Path
from dotenv import set_key
import structlog

# ---------------------------------------------------------------------
# 1) Setup and context
# ---------------------------------------------------------------------
PROJECT_ROOT = Path.cwd()
RAW_DIR = PROJECT_ROOT / "data" / "raw"
ENV_PATH = PROJECT_ROOT / ".env"

log = structlog.get_logger("wildfiresai").bind(stage="date_sync")

# ---------------------------------------------------------------------
# 2) Detect temporal window from processed FIRMS datasets 
# ---------------------------------------------------------------------

from datetime import datetime

# Allow user override via environment variable
FIRMS_DIR = Path(os.getenv("FIRMS_DIR", PROJECT_ROOT / "data" / "processed"))

# Match both CSV and Parquet, flexible names (e.g. fires_terrain_global_20251021.parquet)
patterns = ["fires_*_*.parquet", "fires_*_*.csv", "firms_*_*.parquet", "firms_*_*.csv"]
firms_files = []
for pat in patterns:
    firms_files += list(FIRMS_DIR.glob(pat))

firms_files = sorted(firms_files, key=lambda p: p.stat().st_mtime)
date_from, date_to = None, None

if firms_files:
    latest = firms_files[-1].name
    import re
    # Capture any 8-digit date pattern in the filename
    m = re.findall(r"\d{8}", latest)
    if m:
        date_to = datetime.strptime(m[-1], "%Y%m%d").date().isoformat()
        if len(m) >= 2:
            date_from = datetime.strptime(m[-2], "%Y%m%d").date().isoformat()
        else:
            # If only one date present → 7-day window
            dt = datetime.strptime(m[-1], "%Y%m%d").date()
            date_from = (dt - timedelta(days=7)).isoformat()


# ---------------------------------------------------------------------
# 3) Default window (7 days, UTC-safe)
# ---------------------------------------------------------------------
if not date_from or not date_to:
    now = datetime.now(timezone.utc).date()
    date_to = now.isoformat()
    date_from = (now - timedelta(days=7)).isoformat()

# ---------------------------------------------------------------------
# 4) Override with environment variables only if detection failed
# ---------------------------------------------------------------------
env_from = os.getenv("WF_DATE_FROM")
env_to = os.getenv("WF_DATE_TO")

# Only use .env values if no dates were detected automatically
if not date_from or not date_to:
    if env_from:
        date_from = env_from
    if env_to:
        date_to = env_to
else:
    # Detected dates take precedence, unless explicitly forced
    log.info("Using detected FIRMS-derived window", date_from=date_from, date_to=date_to)

# ---------------------------------------------------------------------
# 5) Validate and persist (only update .env if changed)
# ---------------------------------------------------------------------
try:
    dt_from = datetime.fromisoformat(date_from)
    dt_to = datetime.fromisoformat(date_to)
    if dt_to < dt_from:
        raise ValueError("WF_DATE_TO earlier than WF_DATE_FROM")
except Exception as e:
    now = datetime.now(timezone.utc).date()
    dt_to = datetime.combine(now, datetime.min.time())
    dt_from = dt_to - timedelta(days=7)
    date_from, date_to = dt_from.date().isoformat(), dt_to.date().isoformat()
    log.warning("Invalid date window detected, reset to 7 days", error=str(e))

os.environ["WF_DATE_FROM"] = date_from
os.environ["WF_DATE_TO"] = date_to

# Update .env only if necessary (avoids write churn in clusters)
for k, v in {"WF_DATE_FROM": date_from, "WF_DATE_TO": date_to}.items():
    current = os.getenv(k)
    if current != v:
        set_key(str(ENV_PATH), k, v)

# ---------------------------------------------------------------------
# 6) Context summary + logging
# ---------------------------------------------------------------------
mode = os.getenv("FGPL_MODE", "GLOBAL").upper()
region = os.getenv("WF_REGION", "GLOBAL").upper()
window_days = (datetime.fromisoformat(date_to) - datetime.fromisoformat(date_from)).days

summary = {
    "mode": mode,
    "region": region,
    "WF_DATE_FROM": date_from,
    "WF_DATE_TO": date_to,
    "window_days": window_days,
    "detected_from": firms_files[-1].name if firms_files else "none",
    "timestamp_utc": datetime.now(timezone.utc).isoformat(),
}

log.info("Smart date synchronization complete", **summary)

print("───────────────────────────────────────────────")
print(" WildfiresAI — Smart Date Synchronization")
print("───────────────────────────────────────────────")
print(f"Mode: {mode} | Region: {region}")
print(f"WF_DATE_FROM = {date_from}")
print(f"WF_DATE_TO   = {date_to}")
print(f"Active window: {window_days} days")
if firms_files:
    print(f"Detected from local FIRMS file: {firms_files[-1].name}")
else:
    print("No local FIRMS data found — using default 7-day window.")
print(f"Timestamp (UTC): {summary['timestamp_utc']}")
print("───────────────────────────────────────────────")


{"stage": "date_sync", "date_from": "2025-10-28", "date_to": "2025-11-04", "event": "Using detected FIRMS-derived window", "timestamp": "2025-11-04T18:29:29.704296Z", "level": "info"}
{"stage": "date_sync", "mode": "GLOBAL", "region": "GLOBAL", "WF_DATE_FROM": "2025-10-28", "WF_DATE_TO": "2025-11-04", "window_days": 7, "detected_from": "fires_terrain_EUROPE_2025-10-21_2025-10-28.parquet", "timestamp_utc": "2025-11-04T18:29:29.705465+00:00", "event": "Smart date synchronization complete", "timestamp": "2025-11-04T18:29:29.705516Z", "level": "info"}
───────────────────────────────────────────────
 WildfiresAI — Smart Date Synchronization
───────────────────────────────────────────────
Mode: GLOBAL | Region: GLOBAL
WF_DATE_FROM = 2025-10-28
WF_DATE_TO   = 2025-11-04
Active window: 7 days
Detected from local FIRMS file: fires_terrain_EUROPE_2025-10-21_2025-10-28.parquet
Timestamp (UTC): 2025-11-04T18:29:29.705465+00:00
───────────────────────────────────────────────


# -------------------------------------------------------------------------
# ==== WildfiresAI — Cell 3.2: OpenAI API Integration (Global Adaptive) ====

Purpose:
    - Establish secure global connection to the OpenAI API.
    - Adapt automatically between ONLINE and OFFLINE modes.
    - Expose consistent defaults (model, temperature) for all LLM-backed modules.
# -------------------------------------------------------------------------


In [6]:
# ====  Cell 3.2: OpenAI API Integration (Global Adaptive, Lab-grade) ====


from __future__ import annotations
import os, socket
import openai
from dotenv import set_key
import structlog

log = structlog.get_logger("wildfiresai").bind(stage="openai_init")

# ---------------------------------------------------------------------
# 1) Detect API key and connectivity
# ---------------------------------------------------------------------
api_key = os.getenv("OPENAI_API_KEY", "")
if not api_key:
    raise EnvironmentError(
        "OPENAI_API_KEY not found. Export it in your terminal or .env file before continuing."
    )

def online(host: str = "api.openai.com", port: int = 443, timeout: int = 3) -> bool:
    """Return True if outbound connection to OpenAI API is possible."""
    try:
        socket.create_connection((host, port), timeout=timeout)
        return True
    except Exception:
        return False

is_online = online()

# ---------------------------------------------------------------------
# 2) Configure client
# ---------------------------------------------------------------------
openai.api_key = api_key
openai_version = getattr(openai, "__version__", "unknown")

if is_online:
    print(" OpenAI API connection established.")
else:
    print(" No external network detected — switching to OFFLINE mode.")

# ---------------------------------------------------------------------
# 3) Global defaults (adaptive)
# ---------------------------------------------------------------------
DEFAULT_LLM_MODEL = os.getenv("LLM_MODEL", "gpt-5.1" if is_online else "gpt-4o-mini")
DEFAULT_TEMPERATURE = float(os.getenv("LLM_TEMPERATURE", "0.2"))

# Persist only if values changed
env_path = ".env"
current_env = {k: os.getenv(k) for k in ("LLM_MODEL", "LLM_TEMPERATURE")}
if current_env.get("LLM_MODEL") != DEFAULT_LLM_MODEL:
    set_key(env_path, "LLM_MODEL", DEFAULT_LLM_MODEL)
if current_env.get("LLM_TEMPERATURE") != str(DEFAULT_TEMPERATURE):
    set_key(env_path, "LLM_TEMPERATURE", str(DEFAULT_TEMPERATURE))

# ---------------------------------------------------------------------
# 4) Structured logging summary
# ---------------------------------------------------------------------
log.info(
    "OpenAI API configured",
    network="ONLINE" if is_online else "OFFLINE",
    model=DEFAULT_LLM_MODEL,
    temperature=DEFAULT_TEMPERATURE,
    openai_version=openai_version,
)

print("───────────────────────────────────────────────")
print(" WildfiresAI — OpenAI Global Integration")
print("───────────────────────────────────────────────")
print(f"Network: {'ONLINE' if is_online else 'OFFLINE'}")
print(f"Model:   {DEFAULT_LLM_MODEL}")
print(f"Temp:    {DEFAULT_TEMPERATURE}")
print(f"openai-sdk: {openai_version}")
print("───────────────────────────────────────────────")


 OpenAI API connection established.
{"stage": "openai_init", "network": "ONLINE", "model": "gpt-5.1", "temperature": 0.2, "openai_version": "1.107.1", "event": "OpenAI API configured", "timestamp": "2025-11-04T18:29:38.996610Z", "level": "info"}
───────────────────────────────────────────────
 WildfiresAI — OpenAI Global Integration
───────────────────────────────────────────────
Network: ONLINE
Model:   gpt-5.1
Temp:    0.2
openai-sdk: 1.107.1
───────────────────────────────────────────────


# -------------------------------------------------------------------------
# ==== WildfiresAI — Cell 3.3: Global Logging & Scientific Telemetry ====

Purpose:
    - Configure unified logging for all WildfiresAI frameworks.
    - Record every major event, warning, and error with timestamps.
    - Integrate with tqdm progress bars and persist logs to /reports/logs/.
# -------------------------------------------------------------------------

In [7]:
# ==== WildfiresAI — Cell 3.3: Global Logging & Scientific Telemetry (Lab-grade) ====

from __future__ import annotations
import os, sys, logging, structlog
from datetime import datetime, timezone
from pathlib import Path
from tqdm.auto import tqdm

# ---------------------------------------------------------------------
# 1) Define persistent log paths
# ---------------------------------------------------------------------
PROJECT_ROOT = Path.cwd()
LOG_DIR = PROJECT_ROOT / "reports" / "logs"
LOG_DIR.mkdir(parents=True, exist_ok=True)

LOG_FILE = LOG_DIR / f"wildfiresai_{datetime.now(timezone.utc).strftime('%Y%m%d_%H%M%S')}.log"

# ---------------------------------------------------------------------
# 2) Configure Python logging (base + file handler)
# ---------------------------------------------------------------------
file_handler = logging.FileHandler(LOG_FILE, mode="a", encoding="utf-8")
stream_handler = logging.StreamHandler(sys.stdout)

logging.basicConfig(
    level=logging.INFO,
    handlers=[file_handler, stream_handler],
    format="%(message)s"
)

# ---------------------------------------------------------------------
# 3) Configure structlog for unified output
# ---------------------------------------------------------------------
structlog.configure(
    processors=[
        structlog.processors.TimeStamper(fmt="iso", utc=True),
        structlog.stdlib.add_log_level,
        structlog.processors.StackInfoRenderer(),
        structlog.processors.format_exc_info,
        structlog.processors.JSONRenderer()  # JSON-safe format for scientific telemetry
    ],
    wrapper_class=structlog.make_filtering_bound_logger(logging.INFO),
    context_class=dict,
    logger_factory=structlog.stdlib.LoggerFactory(),
)

log = structlog.get_logger("wildfiresai").bind(stage="global_logger")

# ---------------------------------------------------------------------
# 4) tqdm-compatible progress wrapper
# ---------------------------------------------------------------------
def log_progress(iterable, desc: str = "Processing", total: int | None = None):
    """Wrapper around tqdm with safe logging output."""
    return tqdm(iterable, desc=desc, total=total, dynamic_ncols=True, leave=False)

# ---------------------------------------------------------------------
# 5) Unified persistent logging function
# ---------------------------------------------------------------------
def log_event(level: str, message: str, **kwargs) -> None:
    """Unified logger with UTC timestamps and file persistence."""
    ts = datetime.now(timezone.utc).isoformat()
    record = {"timestamp": ts, "level": level.upper(), "message": message, **kwargs}
    if level.lower() == "error":
        log.error(message, **record)
    elif level.lower() == "warning":
        log.warning(message, **record)
    else:
        log.info(message, **record)

# ---------------------------------------------------------------------
# 6) Example test entry
# ---------------------------------------------------------------------
log_event("info", "Global logging system initialized", log_file=str(LOG_FILE))

# ---------------------------------------------------------------------
# 7) User feedback summary
# ---------------------------------------------------------------------
print("───────────────────────────────────────────────")
print("  WildfiresAI — Global Logging System Active")
print("───────────────────────────────────────────────")
print(f"Logs directory: {LOG_DIR}")
print(f"Session log:    {LOG_FILE.name}")
print("───────────────────────────────────────────────")


{"stage": "global_logger", "timestamp": "2025-11-04T18:29:43.769809Z", "level": "info", "message": "Global logging system initialized", "log_file": "/Users/evareysanchez/WildfiresAI/reports/logs/wildfiresai_20251104_182943.log", "event": "Global logging system initialized"}
───────────────────────────────────────────────
  WildfiresAI — Global Logging System Active
───────────────────────────────────────────────
Logs directory: /Users/evareysanchez/WildfiresAI/reports/logs
Session log:    wildfiresai_20251104_182943.log
───────────────────────────────────────────────


# -------------------------------------------------------------------------
## ==== Cell 4 — AG² Framework Architecture Overview ====
 Purpose
This section introduces the AG² (Analysis → Generation → Action) architecture — the core logic driving WildfiresAI.

- **Framework A – Wildfire Intelligence**  
  Gathers and analyzes environmental, climatic, and terrain data (FIRMS, DEM, Open-Meteo) to estimate ignition risk and propagation dynamics.

- **LLM Orchestrator – Natural-Language Bridge**  
  Parses user queries (e.g., “fires in Spain last week”) and translates them into structured AG² contexts 
  (region, timeframe, intent), dynamically triggering the right modules.

- **Coordinator – AG² Bridge**  
  Mediates data flow from A → B, ensuring JSON integrity, spatiotemporal consistency, and versioned reproducibility.

- **Framework B – Materials Intelligence**  
  Consumes A’s outputs and applies materials-science reasoning (via MP API) to identify optimal compounds for fire containment.

All inter-framework communication occurs via structured JSON under `/reports/`, ensuring transparency and traceability of every AG² cycle.
# -------------------------------------------------------------------------



In [8]:
# ==== WildfiresAI — Cell 4: AG² Framework Architecture Overview ====
from __future__ import annotations
from pathlib import Path
import os, json, datetime as dt, pandas as pd
from typing import Optional

# ---------------------------------------------------------------------------
# 1️) Environment & paths
# ---------------------------------------------------------------------------
PROJECT_ROOT  = globals().get("PROJECT_ROOT", Path.cwd())
DATA_DIR      = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR   = PROJECT_ROOT / "reports"

for p in (DATA_DIR, PROCESSED_DIR, REPORTS_DIR):
    p.mkdir(parents=True, exist_ok=True)

WF_REGION     = os.getenv("WF_REGION", "GLOBAL")
WF_DATE_FROM  = os.getenv("WF_DATE_FROM", "unknown_from")
WF_DATE_TO    = os.getenv("WF_DATE_TO", "unknown_to")

# ---------------------------------------------------------------------------
# 2️) Helper utilities
# ---------------------------------------------------------------------------
def _latest_file_glob(pattern: str, base: Path) -> Optional[Path]:
    files = sorted(base.glob(pattern), key=lambda p: p.stat().st_mtime, reverse=True)
    return files[0] if files else None

def _dump_report(name: str, text_payload: str) -> Path:
    """Persist any text/JSON payload under reports/<name>.json."""
    path = REPORTS_DIR / f"{name}.json"
    try:
        obj = json.loads(text_payload) if text_payload.strip().startswith("{") else {"payload": text_payload}
    except Exception:
        obj = {"payload": text_payload}
    path.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")
    return path

# ---------------------------------------------------------------------------
# 3️) Coordinator — AG² Bridge (A ↔ B)
# ---------------------------------------------------------------------------
class Coordinator:
    """
    Core bridge between Framework A, Framework B, and now the LLM Orchestrator.

    - Executes A → collects wildfire intelligence.
    - Passes A’s JSON payload into B for materials or containment reasoning.
    - Allows partial or full pipeline execution (A only, B only, or A→B).
    - Persists all intermediate results in /reports for full reproducibility.
    """

    def __init__(self):
        self.last_A: Optional[str] = None
        self.last_B: Optional[str] = None
        self.started_at = dt.datetime.now().isoformat(timespec="seconds")

    # --- Execute Framework A (defined in Cell 4.1) ---
    def run_A(self, **kw) -> str:
        from framework_a import run_wildfire_framework  # dynamically resolved
        self.last_A = run_wildfire_framework(**kw)
        print("A →", self.last_A[:200], "...")
        return self.last_A

    # --- Execute Framework B (defined in Cell 4.4) ---
    def run_B(self) -> str:
        from framework_b import run_material_framework  # dynamically resolved
        if not self.last_A:
            print("⚠️ Framework B requested but A not available — running fallback.")
            self.last_A = "{}"
        self.last_B = run_material_framework(self.last_A)
        print("B →", self.last_B[:200], "...")
        return self.last_B

    # --- Full pipeline A→B ---
    def pipeline(self, **kw) -> tuple[str, str]:
        print("══════════════════════════════════════════════════════════")
        print("  WildfiresAI AG² Pipeline (A → B) — Start")
        print("══════════════════════════════════════════════════════════")
        a_summary = self.run_A(**kw)
        b_summary = self.run_B()
        print("══════════════════════════════════════════════════════════")
        print("  WildfiresAI AG² Pipeline Completed")
        print("══════════════════════════════════════════════════════════")
        return a_summary, b_summary

# ---------------------------------------------------------------------------
# 4️) Initialize coordinator
# ---------------------------------------------------------------------------
coordinator = Coordinator()
print(f"Coordinator initialized at {coordinator.started_at}")
print("Ready to link Framework A (4.1) ↔ LLM Orchestrator (4.3) ↔ Framework B (4.4) via AG².")


Coordinator initialized at 2025-11-04T19:29:51
Ready to link Framework A (4.1) ↔ LLM Orchestrator (4.3) ↔ Framework B (4.4) via AG².


# -------------------------------------------------------------------------
# ==== WildfiresAI - Cell 4.0 — Region Selector (NASA FIRMS Feeds) ====
# -------------------------------------------------------------------------


This cell defines the **active geographical region** for WildfiresAI.  
The region determines which **NASA FIRMS dataset** will be used by Framework A
for wildfire detection and environmental analysis.

Available regions:
- `EUROPE`
- `NORTH_AMERICA`
- `SOUTH_AMERICA`
- `AFRICA`
- `ASIA`
- `OCEANIA`
- `GLOBAL`

The system dynamically builds the FIRMS feed URL for the chosen region and 
stores it in environment variables (`WF_REGION`, `FIRMS_VIIRS_<REGION>_7D`).

To switch regions:
```python
WF_REGION = "AFRICA"      # or EUROPE, ASIA, GLOBAL, etc.

# -------------------------------------------------------------------------


In [9]:
# ==== WildfiresAI — Auto Region Selector (All FIRMS Regions, Coordinate-Accurate) ====
import os

# ---------------------------------------------------------------------------
# 1️ Available Regions (NASA FIRMS coordinate-based areas)
# ---------------------------------------------------------------------------
BBOXES = {
    "GLOBAL": "-180.0,-90.0,180.0,90.0",
    "EUROPE": "-25.0,33.0,45.0,72.0",
    "NORTH_AMERICA": "-170.0,5.0,-52.0,83.0",
    "SOUTH_AMERICA": "-93.0,-60.0,-30.0,15.0",
    "ASIA": "25.0,-10.0,180.0,80.0",
    "AFRICA": "-20.0,-35.0,55.0,38.0",
    "OCEANIA": "110.0,-50.0,180.0,0.0",
}

# ---------------------------------------------------------------------------
# 2️ Choose active region (default = EUROPE)
# ---------------------------------------------------------------------------
WF_REGION = os.getenv("WF_REGION", "EUROPE").upper()
if WF_REGION not in BBOXES:
    raise ValueError(f"Invalid WF_REGION '{WF_REGION}'. Must be one of: {list(BBOXES.keys())}")

bbox = BBOXES[WF_REGION]
os.environ["WF_REGION"] = WF_REGION

# ---------------------------------------------------------------------------
# 3️ Build NASA FIRMS feed URL dynamically (7-day VIIRS)
# ---------------------------------------------------------------------------
token = "27f8d7a213b737284b155923ba7dd642"  # Registered FIRMS token
feed_url = f"https://firms.modaps.eosdis.nasa.gov/api/area/csv/{token}/VIIRS_SNPP_NRT/{bbox}/7"

env_key = f"FIRMS_VIIRS_{WF_REGION}_7D"
os.environ[env_key] = feed_url

# ---------------------------------------------------------------------------
# 4️ Feedback summary
# ---------------------------------------------------------------------------
print("───────────────────────────────────────────────")
print(" WildfiresAI — Active FIRMS Region Selected")
print("───────────────────────────────────────────────")
print(f"Region: {WF_REGION}")
print(f"Bounding Box: {bbox}")
print(f"Feed URL: {feed_url}")
print("───────────────────────────────────────────────")


───────────────────────────────────────────────
 WildfiresAI — Active FIRMS Region Selected
───────────────────────────────────────────────
Region: EUROPE
Bounding Box: -25.0,33.0,45.0,72.0
Feed URL: https://firms.modaps.eosdis.nasa.gov/api/area/csv/27f8d7a213b737284b155923ba7dd642/VIIRS_SNPP_NRT/-25.0,33.0,45.0,72.0/7
───────────────────────────────────────────────


# -------------------------------------------------------------------------
## ==== Cell 4.1 — Framework A (Wildfire Intelligence Layer) ====
Framework A = Wildfire Intelligence Layer  

Responsible for environmental analysis, risk estimation, terrain enrichment  
and strategic planning.  
Each agent follows the AG2-style contract (text in → text out JSON string).

Includes the **WildfireFilter** (Universal WildfiresAI Filter, UWF)  
for spatial, temporal, environmental, and scientific filtering.
# -------------------------------------------------------------------------


In [ ]:
# ==== Cell 4.1 — Framework A (Wildfire Intelligence Layer) ====

from __future__ import annotations
from pathlib import Path
from typing import Dict, Any, Optional
import os, json, datetime as dt, pandas as pd, numpy as np, geopandas as gpd
import io, requests
from shapely.geometry import Point

# ---------------------------------------------------------------------------
# 1) Automatically select FIRMS feed based on active region
# ---------------------------------------------------------------------------
region = os.getenv("WF_REGION", "GLOBAL").upper()
feed_key = f"FIRMS_VIIRS_{region}_7D"
FIRMS_CSV_URL = os.getenv(feed_key)

if not FIRMS_CSV_URL:
    raise RuntimeError(f"No FIRMS environment variable found for region '{region}'.")

print(f"Using FIRMS feed: {feed_key}")
print(f"URL: {FIRMS_CSV_URL}")

## ---------------------------------------------------------------------------
# 2) Download the CSV from NASA FIRMS API (with EU fallback if regional empty)
# ---------------------------------------------------------------------------
try:
    # Primary request (regional feed from FIRMS_CSV_URL)
    r = requests.get(FIRMS_CSV_URL, timeout=30)
    r.raise_for_status()
    df_firms = pd.read_csv(io.StringIO(r.text))
    print(f"FIRMS CSV downloaded successfully — {len(df_firms):,} fire detections.")
except Exception as e:
    raise RuntimeError(f"Error downloading or reading FIRMS feed: {e}")

# --- BEGIN PATCH 4.1: EUROPE fallback when regional feed is empty/invalid ---
def _europe_bbox() -> tuple[float, float, float, float]:
    # Extended Europe bounding box: (min_lon, min_lat, max_lon, max_lat)
    return (-31.3, 27.0, 39.7, 71.2)

def _fallback_global_europe() -> pd.DataFrame:
    """
    Fallback to the GLOBAL 7D feed and constrain to Europe bbox.
    Requires FIRMS_VIIRS_GLOBAL_7D in the environment.
    """
    world_url = os.getenv("FIRMS_VIIRS_GLOBAL_7D")
    if not world_url:
        raise RuntimeError("GLOBAL fallback requested but FIRMS_VIIRS_GLOBAL_7D is not set.")
    rr = requests.get(world_url, timeout=60)
    rr.raise_for_status()
    tmp = pd.read_csv(io.StringIO(rr.text))
    tmp = tmp.rename(columns=str.lower)
    if not {"latitude", "longitude"}.issubset(set(tmp.columns)):
        raise RuntimeError("GLOBAL fallback missing latitude/longitude columns.")
    xmin, ymin, xmax, ymax = _europe_bbox()
    tmp = tmp[(tmp["longitude"].between(xmin, xmax)) & (tmp["latitude"].between(ymin, ymax))]
    return tmp.reset_index(drop=True)

# Trigger fallback if the regional response is empty or lacks lat/lon
_need_fallback = (df_firms.shape[0] == 0) or (df_firms.shape[1] == 0)
if not _need_fallback:
    cols_lc = {c.lower() for c in df_firms.columns}
    if not {"latitude", "longitude"}.issubset(cols_lc):
        _need_fallback = True

if _need_fallback:
    print("[FIRMS] Regional feed is empty/invalid. Falling back to GLOBAL feed filtered to Europe.")
    df_firms = _fallback_global_europe()
    print(f"[FIRMS] Fallback rows after Europe bbox: {len(df_firms):,}")

# ---------------------------------------------------------------------------
# 3) Validate and clean essential columns
# ---------------------------------------------------------------------------
expected_cols = ["latitude", "longitude", "brightness", "acq_date", "acq_time", "confidence", "instrument"]
missing = [c for c in expected_cols if c not in df_firms.columns]
if missing:
    print(f"Missing columns in FIRMS data: {missing} (filled with NaN if required)")

df_firms = df_firms.rename(columns=str.lower)
df_firms["datetime"] = pd.to_datetime(
    df_firms["acq_date"] + " " + df_firms["acq_time"].astype(str).str.zfill(4),
    errors="coerce"
)
df_firms["confidence"] = pd.to_numeric(df_firms.get("confidence", 0), errors="coerce")

# ---------------------------------------------------------------------------
# 3.1) Reconstruct brightness if missing (using VIIRS TI4 / TI5 channels)
# ---------------------------------------------------------------------------
if "brightness" not in df_firms.columns:
    if {"bright_ti4", "bright_ti5"}.issubset(df_firms.columns):
        df_firms["brightness"] = (
            0.7 * df_firms["bright_ti4"].astype(float)
            + 0.3 * df_firms["bright_ti5"].astype(float)
        )
        print("[Patch] brightness reconstructed from bright_ti4 + bright_ti5.")
    elif "bright_ti4" in df_firms.columns:
        df_firms["brightness"] = df_firms["bright_ti4"].astype(float)
        print("[Patch] brightness derived from bright_ti4 only.")
    else:
        df_firms["brightness"] = np.nan
        print("[Patch] brightness missing — filled with NaN.")

# ---------------------------------------------------------------------------
# 3.2) Brightness quality audit
# ---------------------------------------------------------------------------
if "brightness" in df_firms.columns:
    total = len(df_firms)
    valid = df_firms["brightness"].notna().sum()
    pct_valid = 100 * valid / total if total > 0 else 0
    print(f"[Audit] Brightness coverage: {valid:,}/{total:,} rows ({pct_valid:.1f}%) valid.")


# ---------------------------------------------------------------------------
# 4) Quick descriptive summary
# ---------------------------------------------------------------------------
fires_total = len(df_firms)
fires_high_conf = len(df_firms[df_firms.get("confidence", 0) > 80])

for col in ["brightness", "bright_ti4", "bright_ti5", "frp"]:
    if col in df_firms.columns:
        mean_brightness = round(df_firms[col].mean(), 2)
        break
else:
    mean_brightness = None

print(
    f"Total fires: {fires_total:,} | High confidence: {fires_high_conf:,} | "
    f"Mean brightness: {mean_brightness}"
)

# ---------------------------------------------------------------------------
# 4.1) Optional Data Integrity Enhancements (non-destructive)
# ---------------------------------------------------------------------------

# 1. Add region/source column for future multi-region integration
df_firms["source_region"] = region

# 2. Drop potential duplicate detections (same lat/lon/date)
before = len(df_firms)
df_firms = df_firms.drop_duplicates(subset=["latitude", "longitude", "acq_date"])
after = len(df_firms)
if before != after:
    print(f"[Clean] Removed {before - after:,} duplicate detections (spatial-temporal).")

# 3. Ensure deterministic column ordering (for reproducible parquet exports)
cols = sorted(df_firms.columns)
df_firms = df_firms[cols]
print("[Meta] DataFrame columns standardized and region tag applied.")


# ---------------------------------------------------------------------------
# 5) Save processed data for Framework A → Coordinator bridge
# ---------------------------------------------------------------------------
PROCESSED_DIR = Path("data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
region_clean = region.lower().replace(" ", "_")

out_path = PROCESSED_DIR / f"fires_terrain_{region_clean}_{pd.Timestamp.now():%Y%m%d}.parquet"
df_firms.to_parquet(out_path, index=False)
print(f"FIRMS data saved to: {out_path}")

# ---------------------------------------------------------------------------
# Shared paths & environment
# ---------------------------------------------------------------------------
PROJECT_ROOT = globals().get("PROJECT_ROOT", Path.cwd())
DATA_DIR      = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR   = PROJECT_ROOT / "reports"
for d in (DATA_DIR, PROCESSED_DIR, REPORTS_DIR):
    d.mkdir(parents=True, exist_ok=True)

WF_REGION = os.getenv("WF_REGION", "GLOBAL")
DATE_FROM = os.getenv("WF_DATE_FROM", "unknown_from")
DATE_TO   = os.getenv("WF_DATE_TO", "unknown_to")

# ---------------------------------------------------------------------------
# Real data ingestion: FIRMS + Open-Meteo + DEM (OpenTopography, COP30 tiling)
# ---------------------------------------------------------------------------
try:
    # Select active FIRMS source (env fallback to region feed)
    firms_env = os.getenv("FIRMS_CSV_URL") or os.getenv(f"FIRMS_VIIRS_{WF_REGION}_7D")
    if not firms_env:
        raise FileNotFoundError("No valid FIRMS data source defined in environment.")
    os.environ["FIRMS_CSV_URL"] = firms_env  # make available to downstream modules

    # Local cache
    raw_dir = DATA_DIR / "raw"
    raw_dir.mkdir(parents=True, exist_ok=True)
    firms_local = raw_dir / f"firms_viirs_snpp_{WF_REGION}_{DATE_FROM}_{DATE_TO}.csv"

    if firms_local.exists():
        df_firms = pd.read_csv(firms_local)
        print(f"[Cache] Using local FIRMS file: {firms_local.name}")
    else:
        headers = {}
        token = os.getenv("NASA_FIRMS_TOKEN", "")
        if token:
            headers["Authorization"] = f"Bearer {token}"
        r = requests.get(firms_env, headers=headers, timeout=60)
        r.raise_for_status()
        firms_local.write_text(r.text, encoding="utf-8")
        df_firms = pd.read_csv(io.StringIO(r.text))
        print(f"[Download] FIRMS data fetched from: {firms_env}")

    # Normalize columns and build GeoDataFrame
    df_firms.rename(columns={"latitude": "lat", "longitude": "lon"}, inplace=True)
    df_firms["acq_date"] = pd.to_datetime(df_firms["acq_date"], errors="coerce")
    gdf_firms = gpd.GeoDataFrame(
        df_firms, geometry=gpd.points_from_xy(df_firms.lon, df_firms.lat), crs="EPSG:4326"
    )

    # ---- Weather join (Open-Meteo)
    lat_c, lon_c = gdf_firms.lat.mean(), gdf_firms.lon.mean()
    r_weather = requests.get(
        "https://archive-api.open-meteo.com/v1/archive",
        params={
            "latitude": lat_c, "longitude": lon_c,
            "start_date": DATE_FROM, "end_date": DATE_TO,
            "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m",
            "timezone": "UTC",
        },
        timeout=30,
    )
    r_weather.raise_for_status()
    w = pd.DataFrame(r_weather.json()["hourly"])
    w["time"] = pd.to_datetime(w["time"])
    w["date"] = w["time"].dt.floor("D")
    df_firms["date"] = df_firms["acq_date"].dt.floor("D")
    df_join = pd.merge(df_firms, w.groupby("date").mean(numeric_only=True), on="date", how="left")
    df_join.rename(
        columns={
            "temperature_2m": "temperature",
            "relative_humidity_2m": "humidity",
            "wind_speed_10m": "wind_ms",
        },
        inplace=True,
    )

    # ---- DEM enrichment (OpenTopography, COP30, 1°×1° tiling)
    try:
        import rasterio

        # Initialize elevation/slope as NaN; will be filled per tile
        df_join["elevation_m"] = np.nan
        df_join["slope_deg"] = np.nan

        # Bin points into 1x1 degree tiles
        df_join["lat_bin"] = np.floor(df_join["lat"]).astype(int)
        df_join["lon_bin"] = np.floor(df_join["lon"]).astype(int)

        # Prioritize tiles with more points to minimize requests
        tile_counts = (
            df_join.groupby(["lat_bin", "lon_bin"]).size().reset_index(name="n").sort_values("n", ascending=False)
        )

        # Safety cap: avoid too many remote calls in one run
        MAX_TILES = 30  # increase if you want higher coverage
        processed_tiles = 0

        for _, row in tile_counts.iterrows():
            if processed_tiles >= MAX_TILES:
                break

            latb = int(row["lat_bin"])
            lonb = int(row["lon_bin"])

            # Clamp latitude to plausible DEM coverage
            south = max(latb, -89)
            north = min(latb + 1, 90)

            # Normalize longitude to [-180, 180]
            west = lonb
            if west < -180:
                west += 360
            if west > 180:
                west -= 360
            east = west + 1
            if east > 180:
                east -= 360

            dem_url = (
                "https://portal.opentopography.org/API/globaldem?"
                f"demtype=COP30&south={south:.6f}&north={north:.6f}"
                f"&west={west:.6f}&east={east:.6f}"
                f"&outputFormat=GTiff&API_Key={os.getenv('OPENTOPO_API_KEY','')}"
            )

            try:
                r = requests.get(dem_url, timeout=60)
                r.raise_for_status()
                if len(r.content) < 8192:
                    # Skip obviously invalid responses
                    continue

                # Persist tile and sample elevations
                tile_path = (DATA_DIR / "raw" / f"dem_tile_{south}_{west}.tif")
                tile_path.write_bytes(r.content)

                # Subset rows in this tile
                mask_tile = (df_join["lat_bin"] == latb) & (df_join["lon_bin"] == lonb)
                sub = df_join.loc[mask_tile, ["lon", "lat"]].to_numpy()

                # Sample elevation from the tile
                with rasterio.open(tile_path) as dem:
                    samples = list(dem.sample(sub))
                elev = np.array([float(v[0]) if (v is not None and np.isfinite(v[0])) else np.nan for v in samples])

                # Write back elevations for those indices
                df_join.loc[mask_tile, "elevation_m"] = elev

                # Slope left as NaN (to be computed later using neighborhood kernels if required)
                processed_tiles += 1

            except Exception:
                # Skip this tile and continue with the next most-populated one
                continue

        print(f"[DEM] processed_tiles={processed_tiles}")
        # Clean up helper bins
        df_join.drop(columns=["lat_bin", "lon_bin"], inplace=True)

    except Exception as e:
        print("DEM enrichment skipped:", e)
        df_join["elevation_m"] = np.nan
        df_join["slope_deg"] = np.nan

    # ---- Vegetation index proxy (placeholder)
    df_join["veg_index"] = np.clip(np.random.normal(0.6, 0.15, len(df_join)), 0, 1)

    # ---- Ensure geometry column exists before saving
    df_join["geometry"] = gpd.points_from_xy(df_join.lon, df_join.lat)

    # ---- Persist processed data
    out_path = PROCESSED_DIR / f"fires_terrain_{WF_REGION}_{DATE_FROM}_{DATE_TO}.parquet"
    gdf_final = gpd.GeoDataFrame(df_join, geometry="geometry", crs="EPSG:4326")
    gdf_final.to_parquet(out_path, index=False)
    print(f"[Data] FIRMS + Meteo + DEM data written to {out_path}")

except Exception as e:
    print("Data ingestion failed:", e)


# ---------------------------------------------------------------------------
# WildfireFilter — Universal Filter System (UWF)
# ---------------------------------------------------------------------------
class WildfireFilter:
    """Universal Wildfire Filter with region/time/environment/confidence filters."""
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()

    def by_region(self, region: Optional[str] = None, bbox: Optional[tuple] = None,
                  radius_km: Optional[float] = None, center: Optional[tuple] = None):
        if bbox:
            xmin, ymin, xmax, ymax = bbox
            self.df = self.df[
                (self.df["lon"] >= xmin) & (self.df["lon"] <= xmax) &
                (self.df["lat"] >= ymin) & (self.df["lat"] <= ymax)
            ]
        elif radius_km and center:
            gdf = gpd.GeoDataFrame(
                self.df, geometry=gpd.points_from_xy(self.df.lon, self.df.lat), crs="EPSG:4326"
            ).to_crs(epsg=3857)
            cx, cy = gpd.GeoSeries([Point(center)], crs="EPSG:4326").to_crs(epsg=3857).iloc[0].coords[0]
            self.df["dist_m"] = gdf.geometry.distance(Point(cx, cy))
            self.df = self.df[self.df["dist_m"] <= radius_km * 1000]
        elif region and "region" in self.df.columns:
            self.df["region_match"] = self.df["region"].astype(str).str.contains(region, case=False, na=False)
            self.df = self.df[self.df["region_match"]]
        return self

    def by_time(self, start: Optional[str] = None, end: Optional[str] = None, days: Optional[int] = None):
        if "acq_date" not in self.df.columns:
            return self
        self.df["acq_date"] = pd.to_datetime(self.df["acq_date"], errors="coerce")
        if days:
            end_dt = pd.Timestamp.now()
            start_dt = end_dt - pd.Timedelta(days=days)
        else:
            start_dt = pd.to_datetime(start) if start else self.df["acq_date"].min()
            end_dt   = pd.to_datetime(end)   if end   else self.df["acq_date"].max()
        self.df = self.df[(self.df["acq_date"] >= start_dt) & (self.df["acq_date"] <= end_dt)]
        return self

    def by_environment(self, temp: Optional[float] = None, humidity: Optional[float] = None,
                       slope: Optional[float] = None):
        if temp is not None and "temperature" in self.df.columns:
            self.df = self.df[self.df["temperature"] >= float(temp)]
        if humidity is not None and "humidity" in self.df.columns:
            self.df = self.df[self.df["humidity"] <= float(humidity)]
        if slope is not None and "slope_deg" in self.df.columns:
            self.df = self.df[self.df["slope_deg"] >= float(slope)]
        return self

    def by_confidence(self, min_level: str = "nominal"):
        if "confidence" not in self.df.columns:
            return self
        mapping = {"low": 1, "nominal": 2, "high": 3}
        self.df["conf_num"] = self.df["confidence"].map(mapping).fillna(0)
        self.df = self.df[self.df["conf_num"] >= mapping.get(min_level, 2)]
        return self

    def by_frp(self, min_mw: float = 10):
        if "frp" in self.df.columns:
            self.df = self.df[self.df["frp"] >= float(min_mw)]
        return self

    def combine(self) -> pd.DataFrame:
        return self.df.reset_index(drop=True)

# ---------------------------------------------------------------------------
# Base interface for text agents
# ---------------------------------------------------------------------------
class TextAgent:
    """Minimal AG2-style interface: text in → text out (JSON string)."""
    def handle(self, text: str = "") -> str:
        raise NotImplementedError

# ---------------------------------------------------------------------------
# AGENTS
# ---------------------------------------------------------------------------
class DataAgentWildfire(TextAgent):
    def _latest(self, pattern: str) -> Optional[Path]:
        files = sorted(PROCESSED_DIR.glob(pattern), key=lambda p: p.stat().st_mtime, reverse=True)
        return files[-1] if files else None

    def _handle_locate(self) -> dict:
        artifacts = {
            "fires_clean": PROCESSED_DIR / f"firms_clean_{WF_REGION}_{DATE_FROM}_{DATE_TO}.csv",
            "fires_terrain": PROCESSED_DIR / f"fires_terrain_{WF_REGION}_{DATE_FROM}_{DATE_TO}.parquet",
            "weather_pts": PROCESSED_DIR / "weather_points.parquet",
        }
        for k, p in artifacts.items():
            if not p.exists():
                artifacts[k] = self._latest(f"{k.split('_')[0]}_*")
        return {k: str(v) if v else None for k, v in artifacts.items()}

    def handle(self, text: str = "") -> str:
        loc = self._handle_locate()
        counts = {}
        try:
            if loc["fires_terrain"] and Path(loc["fires_terrain"]).exists():
                import pyarrow.parquet as pq
                counts["fires_terrain_rows"] = int(pq.read_table(loc["fires_terrain"]).num_rows)
        except Exception:
            pass
        payload = {
            "agent": "DataAgentWildfire",
            "region": WF_REGION,
            "window": {"from": DATE_FROM, "to": DATE_TO},
            "artifacts": loc,
            "counts": counts,
            "timestamp": dt.datetime.utcnow().isoformat(timespec="seconds"),
        }
        return json.dumps(payload, ensure_ascii=False)

class GeoTerrainAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        parquet = PROCESSED_DIR / f"fires_terrain_{WF_REGION}_{DATE_FROM}_{DATE_TO}.parquet"
        if not parquet.exists():
            return json.dumps({"agent": "GeoTerrainAgent", "status": "skipped", "reason": "missing parquet"})
        df = pd.read_parquet(parquet)
        stats = {
            "rows": len(df),
            "elev_mean": float(df.get("elevation_m", pd.Series(dtype=float)).mean()),
            "slope_mean": float(df.get("slope_deg", pd.Series(dtype=float)).mean()),
        }
        return json.dumps({"agent": "GeoTerrainAgent", "status": "ok", **stats})

class VegConditionAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        df = pd.read_parquet(PROCESSED_DIR / f"fires_terrain_{WF_REGION}_{DATE_FROM}_{DATE_TO}.parquet")
        mean = float(df["veg_index"].mean())
        return json.dumps({"agent": "VegConditionAgent", "status": "ok", "mean_index": round(mean, 3)})

class HumanActivityAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        activity_score = round(float(np.random.beta(2, 5)), 3)
        return json.dumps({"agent": "HumanActivityAgent", "activity_score": activity_score})

class FireHistoryAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        freq = int(np.random.randint(5, 50))
        return json.dumps({"agent": "FireHistoryAgent", "fires_since_2000": freq})

class AnalogFinderAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        analogs = [{"year": y, "similarity": round(float(np.random.rand()), 2)} for y in range(2015, 2025)]
        return json.dumps({"agent": "AnalogFinderAgent", "analogs": analogs})

class IgnitionRiskAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        df = pd.read_parquet(PROCESSED_DIR / f"fires_terrain_{WF_REGION}_{DATE_FROM}_{DATE_TO}.parquet")
        risk = min(
            1.0,
            max(
                0.0,
                0.02 * float(df["temperature"].mean())
                + 0.001 * float(df["wind_ms"].mean())
                - 0.003 * float(df["humidity"].mean())
                + 0.0005 * float(df["slope_deg"].mean()),
            ),
        )
        return json.dumps({"agent": "IgnitionRiskAgent", "probability_48h": round(risk, 3)})

class ForecastAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        df = pd.read_parquet(PROCESSED_DIR / f"fires_terrain_{WF_REGION}_{DATE_FROM}_{DATE_TO}.parquet")
        wind = float(df["wind_ms"].mean())
        speed = round(0.3 + wind * 0.2, 2)
        direction = np.random.choice(["N", "S", "E", "W", "NE", "NW", "SE", "SW"])
        return json.dumps({"agent": "ForecastAgent", "rate_km_h": speed, "direction": direction})

class AnalysisAgentWildfire(TextAgent):
    def handle(self, text: str = "") -> str:
        df = pd.read_parquet(PROCESSED_DIR / f"fires_terrain_{WF_REGION}_{DATE_FROM}_{DATE_TO}.parquet")
        payload = {
            "agent": "AnalysisAgentWildfire",
            "region": WF_REGION,
            "window": {"from": DATE_FROM, "to": DATE_TO},
            "metrics": {
                "slope_mean_deg": round(float(df["slope_deg"].mean()), 2),
                "elevation_median": round(float(df["elevation_m"].median()), 2),
                "veg_index": round(float(df["veg_index"].mean()), 2),
                "activity_score": round(float(np.random.beta(2, 5)), 2),
                "ignition_risk": round(float(df["temperature"].mean() / 50), 2),
            },
            "artifacts": {"fires_terrain": f"fires_terrain_{WF_REGION}_{DATE_FROM}_{DATE_TO}.parquet"},
            "timestamp": dt.datetime.utcnow().isoformat(timespec="seconds"),
        }
        return json.dumps(payload, ensure_ascii=False)

# ---------------------------------------------------------------------------
# EFFIS Agents — Coordinated Access to Copernicus Emergency Services
# ---------------------------------------------------------------------------
EFFIS_BASE = "https://maps.effis.emergency.copernicus.eu/effis"
EFFIS_WFS_URL = os.getenv("EFFIS_WFS_URL", f"{EFFIS_BASE}/ows")
EFFIS_WMS_URL = os.getenv("EFFIS_WMS_URL", f"{EFFIS_BASE}/ows")
EFFIS_WCS_URL = os.getenv("EFFIS_WCS_URL", f"{EFFIS_BASE}/ows")
EFFIS_TYPENAME = os.getenv("EFFIS_TYPENAME", "ms:modis.ba.poly")

class EFFIS_WFSAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        try:
            params = {
                "service": "WFS", "request": "GetFeature",
                "typename": EFFIS_TYPENAME, "outputFormat": "application/json",
                "version": "1.1.0", "maxFeatures": 1000,
            }
            r = requests.get(EFFIS_WFS_URL, params=params, timeout=60)
            r.raise_for_status()
            out = REPORTS_DIR / "effis_wfs_burnt_areas.geojson"
            out.write_text(r.text, encoding="utf-8")
            gdf = gpd.read_file(out)
            stats = {"features": int(len(gdf)), "bbox": gdf.total_bounds.tolist()}
            return json.dumps({"agent": "EFFIS_WFSAgent", "status": "ok", **stats})
        except Exception as e:
            return json.dumps({"agent": "EFFIS_WFSAgent", "status": "error", "reason": str(e)})

class EFFIS_WMSAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        try:
            yday = (dt.date.today() - dt.timedelta(days=1)).isoformat()
            bbox = "-18,27,42,72"
            params = {
                "SERVICE": "WMS", "REQUEST": "GetMap", "VERSION": "1.1.1",
                "LAYERS": "ecmwf007.fwi", "STYLES": "",
                "SRS": "EPSG:4326", "BBOX": bbox,
                "WIDTH": 1024, "HEIGHT": 768,
                "FORMAT": "image/png", "TRANSPARENT": "true", "TIME": yday,
            }
            r = requests.get(EFFIS_WMS_URL, params=params, timeout=60)
            r.raise_for_status()
            out = REPORTS_DIR / "effis_wms_map.png"
            out.write_bytes(r.content)
            return json.dumps({"agent": "EFFIS_WMSAgent", "status": "ok", "map": str(out)})
        except Exception as e:
            return json.dumps({"agent": "EFFIS_WMSAgent", "status": "error", "reason": str(e)})

class EFFIS_WCSAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        try:
            params = {
                "SERVICE": "WCS", "REQUEST": "GetCoverage", "VERSION": "1.0.0",
                "COVERAGE": "ecmwf007.fwi", "CRS": "EPSG:4326",
                "BBOX": "-18,27,42,72", "FORMAT": "GeoTIFF",
                "WIDTH": 512, "HEIGHT": 512,
            }
            r = requests.get(EFFIS_WCS_URL, params=params, timeout=90)
            r.raise_for_status()
            out = REPORTS_DIR / "effis_wcs_coverage.tif"
            out.write_bytes(r.content)
            return json.dumps({"agent": "EFFIS_WCSAgent", "status": "ok", "coverage": str(out)})
        except Exception as e:
            return json.dumps({"agent": "EFFIS_WCSAgent", "status": "error", "reason": str(e)})

# ---------------------------------------------------------------------------
# LLM Bridge — Natural-Language Routing to Agents
# ---------------------------------------------------------------------------
AGENT_REGISTRY: Dict[str, TextAgent] = {
    "data": DataAgentWildfire(),
    "geo": GeoTerrainAgent(),
    "veg": VegConditionAgent(),
    "activity": HumanActivityAgent(),
    "history": FireHistoryAgent(),
    "analog": AnalogFinderAgent(),
    "risk": IgnitionRiskAgent(),
    "forecast": ForecastAgent(),
    "analysis": AnalysisAgentWildfire(),
    "effis_wfs": EFFIS_WFSAgent(),
    "effis_wms": EFFIS_WMSAgent(),
    "effis_wcs": EFFIS_WCSAgent(),
}

def run_wildfire_framework(
    query: str = "",
    region: str = "GLOBAL",
    date_from: str = "",
    date_to: str = "",
    **kwargs
) -> str:
    """
    Extended version to accept region/date arguments for Coordinator compatibility.
    Keeps backward compatibility with all previous calls.
    """
    # Environment sync (so downstream agents see consistent variables)
    os.environ["WF_REGION"] = region
    os.environ["WF_DATE_FROM"] = date_from
    os.environ["WF_DATE_TO"] = date_to

    q = (query or "").lower()

    if "map" in q or "wms" in q:
        return AGENT_REGISTRY["effis_wms"].handle(q)
    elif "vector" in q or "burnt" in q or "wfs" in q:
        return AGENT_REGISTRY["effis_wfs"].handle(q)
    elif "raster" in q or "ndvi" in q or "wcs" in q:
        return AGENT_REGISTRY["effis_wcs"].handle(q)
    elif "risk" in q:
        return AGENT_REGISTRY["risk"].handle(q)
    elif "forecast" in q:
        return AGENT_REGISTRY["forecast"].handle(q)
    elif "vegetation" in q:
        return AGENT_REGISTRY["veg"].handle(q)
    elif "analysis" in q:
        return AGENT_REGISTRY["analysis"].handle(q)
    else:
        return AGENT_REGISTRY["data"].handle(q)


Using FIRMS feed: FIRMS_VIIRS_EUROPE_7D
URL: https://firms.modaps.eosdis.nasa.gov/api/area/csv/27f8d7a213b737284b155923ba7dd642/VIIRS_SNPP_NRT/-25.0,33.0,45.0,72.0/7
FIRMS CSV downloaded successfully — 5,043 fire detections.
Missing columns in FIRMS data: ['brightness'] (filled with NaN if required)
[Patch] brightness reconstructed from bright_ti4 + bright_ti5.
[Audit] Brightness coverage: 5,043/5,043 rows (100.0%) valid.
Total fires: 5,043 | High confidence: 0 | Mean brightness: 312.23
[Meta] DataFrame columns standardized and region tag applied.
FIRMS data saved to: data/processed/fires_terrain_europe_20251104.parquet
[Cache] Using local FIRMS file: firms_viirs_snpp_EUROPE_2025-10-28_2025-11-04.csv


# -------------------------------------------------------------------------
## ==== Cell 4.2 — AG2 Coordinator Bridge (Framework A ↔ Framework B) ====

This cell defines the **Coordinator**, the core orchestrator of the AG2 architecture.  
It manages execution flow between **Framework A (Wildfire Intelligence)** and **Framework B (Materials Intelligence)**, ensuring:

- Structured JSON payload transfer (text-only communication).  
- Temporal & spatial synchronization of analyses.  
- Full reproducibility via logs and versioned reports in `/reports/`.

The Coordinator enables simple, transparent execution:

```python
coordinator.pipeline(region=WF_REGION, date_from=WF_DATE_FROM, date_to=WF_DATE_TO)

# -------------------------------------------------------------------------


In [11]:
# ==== WildfiresAI — Cell 4.2: AG² Coordinator Bridge ====
from __future__ import annotations

import os, json, traceback, datetime as dt
from pathlib import Path
from typing import Optional, Tuple, Union

# pydantic is strongly recommended; keep graceful degradation if missing
try:
    from pydantic import BaseModel, ValidationError
    HAS_PYDANTIC = True
except Exception:
    HAS_PYDANTIC = False
    BaseModel = object  # minimal fallback


# ---------------------------------------------------------------------------
# 1) Environment & paths
# ---------------------------------------------------------------------------
PROJECT_ROOT = globals().get("PROJECT_ROOT", Path.cwd())
REPORTS_DIR  = PROJECT_ROOT / "reports"
HISTORY_DIR  = REPORTS_DIR / "history"
LOG_FILE     = REPORTS_DIR / "logs" / "coordinator.log"

for d in (REPORTS_DIR, HISTORY_DIR, LOG_FILE.parent):
    d.mkdir(parents=True, exist_ok=True)

WF_REGION     = os.getenv("WF_REGION", "GLOBAL")
WF_DATE_FROM  = os.getenv("WF_DATE_FROM", "unknown_from")
WF_DATE_TO    = os.getenv("WF_DATE_TO", "unknown_to")


# ---------------------------------------------------------------------------
# 2) Structured models for validation
# ---------------------------------------------------------------------------
if HAS_PYDANTIC:
    class SummaryA(BaseModel):
        agent: str = "WildfireFramework"
        region: str
        window: dict
        timestamp: str
        counts: Optional[dict] = None
        signals: Optional[dict] = None

    class SummaryB(BaseModel):
        agent: str = "MaterialsFramework"
        status: str
        wildfire_context: Optional[dict] = None
        candidates_top3: Optional[list] = None
        timestamp: str
else:
    # Light fallbacks (no validation)
    class SummaryA:  # type: ignore
        def __init__(self, **kw): self.__dict__.update(kw)
        def model_dump(self): return self.__dict__
        def model_dump_json(self): return json.dumps(self.__dict__, ensure_ascii=False)

    class SummaryB:  # type: ignore
        def __init__(self, **kw): self.__dict__.update(kw)
        def model_dump(self): return self.__dict__
        def model_dump_json(self): return json.dumps(self.__dict__, ensure_ascii=False)


# ---------------------------------------------------------------------------
# 3) Helpers
# ---------------------------------------------------------------------------
def _now_iso() -> str:
    return dt.datetime.now(dt.UTC).isoformat(timespec="seconds")

def _looks_like_json(text: str) -> bool:
    if not isinstance(text, str): 
        return False
    t = text.strip()
    return (t.startswith("{") and t.endswith("}")) or (t.startswith("[") and t.endswith("]"))

def _write_json(obj: dict, name: str, origin: str = "manual") -> Path:
    """Persist JSON object under reports/history with timestamp and origin label."""
    ts = dt.datetime.now(dt.UTC).strftime("%Y%m%dT%H%M%S")
    path = HISTORY_DIR / f"{ts}_{origin}_{name}.json"
    path.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")
    return path

def _log(msg: str) -> None:
    """Append logs to reports/logs/coordinator.log (and echo)."""
    timestamp = _now_iso()
    line = f"[{timestamp}] {msg}\n"
    print(line.strip())
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(line)

def _extract_json_from_text(text: str) -> dict:
    """Extract JSON payload from text; if not JSON, wrap as raw_text."""
    try:
        # Allow prefixed logs like "[INFO] {...}"
        if "] " in text and text.strip().startswith("["):
            text = text.split("] ", 1)[1]
        return json.loads(text) if _looks_like_json(text) else {"raw_text": text}
    except Exception:
        return {"raw_text": text}

def _set_env_from_kwargs(kwargs: dict) -> None:
    """Propagate region/window from kwargs into environment for downstream consistency."""
    region = kwargs.get("region")
    date_from = kwargs.get("date_from")
    date_to   = kwargs.get("date_to")
    if region:    os.environ["WF_REGION"] = str(region).upper()
    if date_from: os.environ["WF_DATE_FROM"] = str(date_from)
    if date_to:   os.environ["WF_DATE_TO"]   = str(date_to)


# ---------------------------------------------------------------------------
# 4) Coordinator class (A ↔ B ↔ LLM)
# ---------------------------------------------------------------------------
class Coordinator:
    """AG² orchestrator connecting Framework A ↔ Framework B and LLM Orchestrator."""

    def __init__(self):
        self.run_id = dt.datetime.now(dt.UTC).strftime("%Y%m%dT%H%M%S")
        self.last_A: Optional[SummaryA] = None
        self.last_B: Optional[SummaryB] = None
        _log(f"Coordinator initialized (run_id={self.run_id})")

    # -----------------------------------------------------------------------
    # Execute Framework A
    # -----------------------------------------------------------------------
    def run_A(self, **kwargs) -> SummaryA:
        """
        Execute Framework A (Wildfire Intelligence).
        Expects run_wildfire_framework to return a JSON string compatible with SummaryA.
        """
        _set_env_from_kwargs(kwargs)
        _log("Running Framework A (Wildfire Intelligence)…")

        if "run_wildfire_framework" not in globals():
            raise RuntimeError("Function run_wildfire_framework() not found — execute Cell 4.1 first.")

        text = globals()["run_wildfire_framework"](**kwargs)
        data = _extract_json_from_text(text)

        if HAS_PYDANTIC:
            try:
                model = SummaryA(**data)
            except ValidationError as e:
                _log("SummaryA validation failed; wrapping raw payload.")
                model = SummaryA(
                    agent=data.get("agent", "WildfireFramework"),
                    region=os.getenv("WF_REGION", "GLOBAL"),
                    window={"from": os.getenv("WF_DATE_FROM", "unknown_from"),
                            "to":   os.getenv("WF_DATE_TO", "unknown_to")},
                    timestamp=_now_iso(),
                    counts=data.get("counts"),
                    signals={"validation_error": str(e), "payload": data}
                )
        else:
            model = SummaryA(**data)

        _write_json(model.model_dump() if HAS_PYDANTIC else model.model_dump(), "summary_A")
        self.last_A = model
        _log("Framework A OK.")
        return model

    # -----------------------------------------------------------------------
    # Execute Framework B
    # -----------------------------------------------------------------------
    def run_B(self, input_data: Union[SummaryA, str, None] = None) -> SummaryB:
        """
        Execute Framework B (Materials Intelligence).
        Expects run_material_framework(payload_json_str) to exist in Cell 4.4.
        """
        _log("Running Framework B (Materials Intelligence)…")

        if "run_material_framework" not in globals():
            raise RuntimeError("Function run_material_framework() not found — execute Cell 4.4 first.")

        if input_data is None:
            if not self.last_A:
                raise RuntimeError("Framework A must run before Framework B.")
            payload = self.last_A.model_dump_json() if HAS_PYDANTIC else self.last_A.model_dump_json()
        elif isinstance(input_data, SummaryA):
            payload = input_data.model_dump_json() if HAS_PYDANTIC else input_data.model_dump_json()
        else:
            payload = input_data  # already a JSON string

        text = globals()["run_material_framework"](payload)
        data = _extract_json_from_text(text)

        if HAS_PYDANTIC:
            try:
                model = SummaryB(**data)
            except ValidationError as e:
                _log("SummaryB validation failed; wrapping raw payload.")
                model = SummaryB(
                    agent=data.get("agent", "MaterialsFramework"),
                    status=data.get("status", "error"),
                    wildfire_context=data.get("wildfire_context"),
                    candidates_top3=data.get("candidates_top3"),
                    timestamp=_now_iso(),
                )
        else:
            model = SummaryB(**data)

        _write_json(model.model_dump() if HAS_PYDANTIC else model.model_dump(), "summary_B")
        self.last_B = model
        _log("Framework B OK.")
        return model

    # -----------------------------------------------------------------------
    # Standard A→B pipeline
    # -----------------------------------------------------------------------
    def pipeline(self, **kwargs) -> Tuple[SummaryA, SummaryB]:
        """Run A → B with environment propagation and full history."""
        _log("══════════ AG² PIPELINE START ══════════")
        try:
            A = self.run_A(**kwargs)
            B = self.run_B(A)
            _log("AG² pipeline completed successfully.")
            return A, B
        except Exception as e:
            _log("Pipeline failed: " + str(e))
            _log(traceback.format_exc())
            raise
        finally:
            _log("══════════ AG² PIPELINE END ══════════")

    # -----------------------------------------------------------------------
    # LLM integration entrypoints
    # -----------------------------------------------------------------------
    def run_from_orchestrator(self, llm_json: dict) -> SummaryA:
        """Execute Framework A using structured payload from LLM Orchestrator."""
        _log(
            f"Triggered by LLM Orchestrator → region={llm_json.get('region')} "
            f"window={llm_json.get('date_from')}–{llm_json.get('date_to')}"
        )
        model = self.run_A(
            region=llm_json.get("region"),
            date_from=llm_json.get("date_from"),
            date_to=llm_json.get("date_to")
        )
        _write_json(model.model_dump() if HAS_PYDANTIC else model.model_dump(), "summary_A_llm", origin="LLM_ORCHESTRATOR")
        return model

    def pipeline_from_orchestrator(self, llm_json: dict) -> Tuple[SummaryA, SummaryB]:
        """Run full A→B cycle triggered by LLM Orchestrator."""
        _log(f"LLM Orchestrator pipeline start (run_id={self.run_id})")
        A = self.run_from_orchestrator(llm_json)
        B = self.run_B(A)
        _write_json(B.model_dump() if HAS_PYDANTIC else B.model_dump(), "summary_B_llm", origin="LLM_ORCHESTRATOR")
        _log("LLM Orchestrator pipeline completed successfully.")
        return A, B


# ---------------------------------------------------------------------------
# 5) Instantiate Coordinator
# ---------------------------------------------------------------------------
coordinator = Coordinator()
_log("Coordinator ready — linked with Framework A (4.1), LLM Orchestrator (4.3), and Framework B (4.4).")


[2025-11-04T18:36:56+00:00] Coordinator initialized (run_id=20251104T183656)
[2025-11-04T18:36:56+00:00] Coordinator ready — linked with Framework A (4.1), LLM Orchestrator (4.3), and Framework B (4.4).


# -------------------------------------------------------------------------
## ==== Cell 4.3 — LLM Orchestrator Agent (Natural Language → AG² Context) ====

LLM front-end that parses natural-language questions into AG² parameters and dispatches:
- Sets `WF_REGION / WF_DATE_FROM / WF_DATE_TO` in environment for Framework A.
- Routes to EFFIS agents (WMS/WFS/WCS) via `query` when requested.
- Runs A only, B only (if A exists), or A→B pipeline (manual sequencing to avoid kwargs mismatch).

All outputs follow the AG² "text-in → text-out (JSON)" contract.
# -------------------------------------------------------------------------


In [12]:
# ==== WildfiresAI — Cell 4.3: LLM Orchestrator (Natural-Language Reasoning Layer) ====

from __future__ import annotations
import os, re, json, datetime as dt
from typing import Dict, Any, Tuple

# -----------------------------------------------------------------------------
# Lightweight helpers
# -----------------------------------------------------------------------------
def _now_iso() -> str:
    return dt.datetime.utcnow().isoformat(timespec="seconds")

def dump_like(obj) -> dict:
    """Return a dict regardless of pydantic availability."""
    if hasattr(obj, "model_dump"):
        return obj.model_dump()
    if hasattr(obj, "__dict__"):
        return obj.__dict__
    if isinstance(obj, dict):
        return obj
    return {"value": str(obj)}

# -----------------------------------------------------------------------------
# Region and language parsing (EN + ES)
# -----------------------------------------------------------------------------
REGION_KEYWORDS = {
    # ES
    "españa": "SPAIN", "portugal": "PORTUGAL", "francia": "FRANCE",
    "italia": "ITALY", "grecia": "GREECE", "europa": "EUROPE",
    "global": "GLOBAL", "mundo": "GLOBAL", "eeuu": "USA", "estados unidos": "USA",
    # EN
    "spain": "SPAIN", "france": "FRANCE", "italy": "ITALY", "greece": "GREECE",
    "europe": "EUROPE", "usa": "USA", "global": "GLOBAL", "world": "GLOBAL",
}

def _extract_region(text: str) -> str:
    t = text.lower()
    for k, v in REGION_KEYWORDS.items():
        if k in t:
            return v
    return os.getenv("WF_REGION", "GLOBAL").upper()

def _extract_timeframe(text: str) -> Tuple[str, str]:
    """Very simple temporal parsing (ES + EN)."""
    t = text.lower()
    now = dt.datetime.utcnow()
    # ES
    if "ayer" in t:
        start = now - dt.timedelta(days=1)
    elif "última semana" in t or "ultima semana" in t or "semana pasada" in t or "7 días" in t or "7 dias" in t:
        start = now - dt.timedelta(days=7)
    elif "último mes" in t or "ultimo mes" in t or "mes pasado" in t or "30 días" in t or "30 dias" in t:
        start = now - dt.timedelta(days=30)
    # EN
    elif "yesterday" in t:
        start = now - dt.timedelta(days=1)
    elif "past week" in t or "last week" in t or "7 days" in t or "week" in t:
        start = now - dt.timedelta(days=7)
    elif "last month" in t or "past month" in t or "30 days" in t or "month" in t:
        start = now - dt.timedelta(days=30)
    elif re.search(r"\b(20\d{2})\b", t):
        year = int(re.search(r"\b(20\d{2})\b", t).group(1))
        start = dt.datetime(year, 1, 1)
    else:
        start = now - dt.timedelta(days=7)
    return start.strftime("%Y-%m-%d"), now.strftime("%Y-%m-%d")

def _detect_intent(text: str) -> str:
    """Return one of: 'A', 'B', 'AtoB', 'WMS', 'WFS', 'WCS'."""
    t = text.lower()
    # Map/imagery requests → EFFIS WMS
    if any(w in t for w in ["wms", "mapa", "map", "mapa rápido", "quick map"]):
        return "WMS"
    # Vector geometries → EFFIS WFS
    if any(w in t for w in ["wfs", "vector", "burnt", "quemadas", "áreas quemadas", "areas quemadas"]):
        return "WFS"
    # Raster coverage → EFFIS WCS
    if any(w in t for w in ["wcs", "raster", "ndvi", "riesgo", "fwi coverage"]):
        return "WCS"
    # Scientific analysis or forecast → A
    if any(w in t for w in ["forecast", "pronóstico", "pronostico", "risk", "riesgo", "análisis", "analisis"]):
        return "A"
    # Materials/containment focus → AtoB (default)
    if any(w in t for w in ["material", "contain", "gel", "retardant", "retardante"]):
        return "AtoB"
    return "AtoB"

# -----------------------------------------------------------------------------
# Orchestrator
# -----------------------------------------------------------------------------
class LLMOrchestrator:
    """
    Natural-language front-end for AG².
    - Sets environment (WF_REGION/DATE_FROM/DATE_TO) for Framework A.
    - Uses Coordinator to run A only, B only (if available), or manual A→B.
    - Routes EFFIS agents by sending a 'query' that 4.1 understands.
    """

    def __init__(self, coordinator):
        self.coordinator = coordinator
        print("LLM Orchestrator initialized and connected to Coordinator.")

    def parse_query(self, user_text: str) -> Dict[str, Any]:
        region = _extract_region(user_text)
        date_from, date_to = _extract_timeframe(user_text)
        intent = _detect_intent(user_text)
        payload = {
            "region": region,
            "date_from": date_from,
            "date_to": date_to,
            "intent": intent,
            "timestamp": _now_iso(),
        }
        print(f"[LLM Parser] intent={intent} | region={region} | window={date_from}→{date_to}")
        return payload

    def _set_env(self, region: str, date_from: str, date_to: str) -> None:
        """Propagate context to 4.1 via environment (avoids kwargs mismatch)."""
        os.environ["WF_REGION"] = region.upper()
        os.environ["WF_DATE_FROM"] = date_from
        os.environ["WF_DATE_TO"] = date_to

    def execute(self, user_text: str) -> str:
        """
        Main entrypoint:
        - Parses the query.
        - Sets environment.
        - Routes to EFFIS agents or to A/B/A→B as needed.
        Returns a JSON string.
        """
        try:
            parsed = self.parse_query(user_text)
            self._set_env(parsed["region"], parsed["date_from"], parsed["date_to"])
            intent = parsed["intent"]

            # EFFIS direct routes via 4.1 keyword dispatch
            if intent == "WMS":
                a = self.coordinator.run_A(query="wms map")
                return json.dumps({"status": "ok", "stage": "WMS", "summary": dump_like(a)}, ensure_ascii=False, indent=2)
            if intent == "WFS":
                a = self.coordinator.run_A(query="wfs vector burnt")
                return json.dumps({"status": "ok", "stage": "WFS", "summary": dump_like(a)}, ensure_ascii=False, indent=2)
            if intent == "WCS":
                a = self.coordinator.run_A(query="wcs raster ndvi")
                return json.dumps({"status": "ok", "stage": "WCS", "summary": dump_like(a)}, ensure_ascii=False, indent=2)

            # Analysis-only (A), using 4.1's 'analysis' agent
            if intent == "A":
                a = self.coordinator.run_A(query="analysis")
                return json.dumps({"status": "ok", "stage": "A", "summary": dump_like(a)}, ensure_ascii=False, indent=2)

            # Manual A→B pipeline to avoid passing unsupported kwargs to 4.1
            # 1) Run A (analysis agent to ensure meaningful context)
            a = self.coordinator.run_A(query="analysis")
            # 2) Run B if available (4.4); otherwise return A-only with note
            try:
                b = self.coordinator.run_B(a)
                out = {"status": "ok", "stage": "AtoB", "summary_A": dump_like(a), "summary_B": dump_like(b)}
            except Exception as e:
                out = {"status": "partial", "stage": "A", "summary_A": dump_like(a), "note": f"B unavailable: {e}"}
            return json.dumps(out, ensure_ascii=False, indent=2)

        except Exception as e:
            err = {"status": "error", "message": str(e)}
            print("LLM Orchestrator error:", e)
            return json.dumps(err, ensure_ascii=False, indent=2)

# -----------------------------------------------------------------------------
# Instantiate orchestrator
# -----------------------------------------------------------------------------
llm_orchestrator = LLMOrchestrator(coordinator)
print("LLM Orchestrator ready — natural-language queries will be dispatched to AG².")


LLM Orchestrator initialized and connected to Coordinator.
LLM Orchestrator ready — natural-language queries will be dispatched to AG².


-------------------------------------------------------------------------
### Cell 4.4 — Framework B: Materials Intelligence Layer

**Framework B** converts analytical outputs from **Framework A** into actionable intelligence, enabling decision-making and autonomous operations. It includes:

- Context extraction (environmental and physical conditions)
- Real-time material selection** via the *Materials Project API* (no fallback)
- Combinatorial material synthesis (binary/ternary synergy scoring)
- Multi-drone cooperative planning
- Containment simulation and actuation dispatch
- Human-readable mission summaries

All agents follow the AG² text-only contract:

text_in → text_out (JSON)

Logs are written under: /reports/logs/


In [13]:
# ==== WildfiresAI — Cell 4.4: Framework B (Materials Intelligence Layer) ====
from __future__ import annotations
from pathlib import Path
from typing import Dict, Any, Optional, List, Union
import os, json, datetime as dt, numpy as np, traceback, hashlib, itertools

try:
    from mp_api.client import MPRester  # type: ignore
except Exception:
    MPRester = None

# -----------------------------------------------------------------------------
# Environment and logging
# -----------------------------------------------------------------------------
PROJECT_ROOT = globals().get("PROJECT_ROOT", Path.cwd())
REPORTS_DIR  = PROJECT_ROOT / "reports"
LOG_FILE     = REPORTS_DIR / "logs" / "framework_b.log"
for d in (REPORTS_DIR, LOG_FILE.parent):
    d.mkdir(parents=True, exist_ok=True)

MP_API_KEY = os.getenv("MP_API_KEY", "")

def _utcnow() -> str:
    return dt.datetime.now(dt.UTC).isoformat(timespec="seconds")

def _log(msg: str, origin: str = "FrameworkB"):
    line = f"[{_utcnow()}][{origin}] {msg}\n"
    print(line.strip())
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(line)

# -----------------------------------------------------------------------------
# JSON helpers
# -----------------------------------------------------------------------------
def _strip_prefix_if_any(text: str) -> str:
    t = text.strip()
    if t.startswith("[") and "] " in t:
        t = t.split("] ", 1)[1]
    return t

def _safe_json(text: Union[str, dict]) -> dict:
    if isinstance(text, dict):
        return text
    try:
        t = _strip_prefix_if_any(text)
        return json.loads(t) if t.lstrip().startswith("{") else {}
    except Exception:
        return {}

def _get_signal(js: dict, key: str, default: Optional[float] = None) -> Optional[float]:
    sigs = js.get("signals") or {}
    mets = js.get("metrics")  or {}
    val = sigs.get(key, mets.get(key, default))
    return float(val) if isinstance(val, (int, float, np.floating)) else default

# -----------------------------------------------------------------------------
# Base agent
# -----------------------------------------------------------------------------
class TextAgent:
    def handle(self, text: str = "") -> str:
        raise NotImplementedError

# -----------------------------------------------------------------------------
# 1) MaterialsContextBuilder — dynamic weights from wildfire context
# -----------------------------------------------------------------------------
class MaterialsContextBuilder(TextAgent):
    def handle(self, text: str = "") -> str:
        js = _safe_json(text)
        region = js.get("region", js.get("window", {}).get("region", "unknown"))

        temp_c   = _get_signal(js, "temperature", None)
        humid    = _get_signal(js, "humidity", None)
        wind_ms  = _get_signal(js, "wind_ms", None)
        slope    = _get_signal(js, "slope_mean_deg", 10.0)
        veg_idx  = _get_signal(js, "veg_index", None)

        w1, w2, w3 = 0.4, 0.3, 0.3

        if temp_c is not None and temp_c >= 42:
            w1 += 0.1; w3 += 0.05; w2 -= 0.15
        if wind_ms is not None and wind_ms >= 6:
            w2 += 0.1; w1 -= 0.05
        if slope is not None and slope >= 10:
            w2 += 0.05
        if humid is not None and humid >= 70:
            w3 += 0.1; w1 -= 0.05
        if veg_idx is not None and veg_idx >= 0.6:
            w3 += 0.05

        s = max(w1 + w2 + w3, 1e-9)
        w1, w2, w3 = w1 / s, w2 / s, w3 / s

        context = {
            "agent": "MaterialsContextBuilder",
            "status": "ok",
            "context": {
                "region": region,
                "temperature_C": float(temp_c) if temp_c is not None else None,
                "humidity_%": float(humid) if humid is not None else None,
                "wind_ms": float(wind_ms) if wind_ms is not None else None,
                "terrain_slope_deg": float(slope) if slope is not None else None,
                "veg_index": float(veg_idx) if veg_idx is not None else None,
                "weights": {"w1": w1, "w2": w2, "w3": w3},
                "target_properties": ["high_melting_point", "low_density", "chem_stability"]
            },
            "timestamp": _utcnow()
        }
        _log(f"Context built: T={temp_c}°C RH={humid}% wind={wind_ms} slope={slope} veg={veg_idx} weights={context['context']['weights']}")
        return json.dumps(context, ensure_ascii=False)

# -----------------------------------------------------------------------------
# 2) MaterialsAgent — real Materials Project query with filters
# -----------------------------------------------------------------------------
class MaterialsAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        if not MP_API_KEY or not MPRester:
            raise RuntimeError("Materials Project API not available — install mp_api and set MP_API_KEY.")

        ctx = _safe_json(text)
        region = ctx.get("context", {}).get("region", "unknown")
        weights = ctx.get("context", {}).get("weights") or {"w1": 0.4, "w2": 0.3, "w3": 0.3}
        exclude_elems = {"Pb", "As", "Cd", "Hg"}

        with MPRester(MP_API_KEY) as mpr:
            docs = mpr.materials.summary.search(
                density=(None, 4.0),
                energy_above_hull=(None, 0.1),
                formation_energy_per_atom=(-10.0, -3.0),
                band_gap=(3.0, None),
                elements=["O"],
                deprecated=False,
                fields=[
                    "material_id", "formula_pretty", "density",
                    "formation_energy_per_atom", "energy_above_hull",
                    "band_gap", "elements"
                ],
                chunk_size=200
            )

            # Limit total materials returned to prevent slow streaming
            docs = docs[:120]

        clean = [d for d in docs if not (set(getattr(d, "elements", []) or []) & exclude_elems)]
        if not clean:
            raise RuntimeError("No materials matched filters after toxicity exclusion.")

        def _score_one(d):
            fe = getattr(d, "formation_energy_per_atom", 0.0)
            eh = getattr(d, "energy_above_hull", 0.0)
            rho = getattr(d, "density", 0.0)
            bg = getattr(d, "band_gap", 0.0)
            w1, w2, w3 = weights["w1"], weights["w2"], weights["w3"]
            return (w1 * -fe) + (w2 * -rho) + (w3 * -eh) + (bg * 0.05)

        ranked = sorted(clean, key=_score_one, reverse=True)[:12]
        base_candidates = [{
            "material_id": getattr(d, "material_id", None),
            "formula": getattr(d, "formula_pretty", None),
            "density": getattr(d, "density", None),
            "formation_energy_per_atom": getattr(d, "formation_energy_per_atom", None),
            "energy_above_hull": getattr(d, "energy_above_hull", None),
            "band_gap": getattr(d, "band_gap", None),
            "elements": list(getattr(d, "elements", []) or []),
            "score_single": _score_one(d),
        } for d in ranked]

        payload = {
            "agent": "MaterialsAgent",
            "status": "ok",
            "region": region,
            "weights": weights,
            "base_candidates": base_candidates,
            "timestamp": _utcnow()
        }

        _log(f"MaterialsAgent: {len(base_candidates)} base candidates (real, filtered)")
        return json.dumps(payload, ensure_ascii=False)

# -----------------------------------------------------------------------------
# 2.5) MaterialsCombinerAgent — binary/ternary synergy computation
# -----------------------------------------------------------------------------
class MaterialsCombinerAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        js = _safe_json(text)
        base = js.get("base_candidates") or []
        weights = js.get("weights") or {"w1": 0.4, "w2": 0.3, "w3": 0.3}
        if not base:
            raise RuntimeError("No base candidates provided for combination.")

        def _prop(d, k):
            v = d.get(k, None)
            return float(v) if isinstance(v, (int, float, np.floating)) else np.nan

        def _chem_penalty(elems: set) -> float:
            tox = {"Pb", "As", "Cd", "Hg"}
            return 0.5 if (elems & tox) else 0.0

        combos = []
        for i, j in itertools.combinations(range(len(base)), 2):
            a, b = base[i], base[j]
            elems = set((a.get("elements") or []) + (b.get("elements") or []))
            fe = np.nanmean([_prop(a, "formation_energy_per_atom"), _prop(b, "formation_energy_per_atom")])
            rho = np.nanmean([_prop(a, "density"), _prop(b, "density")])
            eh = np.nanmean([_prop(a, "energy_above_hull"), _prop(b, "energy_above_hull")])
            bg = np.nanmean([_prop(a, "band_gap"), _prop(b, "band_gap")])
            w1, w2, w3 = weights["w1"], weights["w2"], weights["w3"]
            s = (w1 * -fe) + (w2 * -rho) + (w3 * -eh) + (bg * 0.05) - _chem_penalty(elems)
            combos.append({
                "type": "binary",
                "members": [a["formula"], b["formula"]],
                "props": {"formation_energy_per_atom": fe, "density": rho, "energy_above_hull": eh, "band_gap": bg},
                "synergy_score": float(s),
            })

        for i, j, k in itertools.combinations(range(len(base)), 3):
            a, b, c = base[i], base[j], base[k]
            elems = set((a.get("elements") or []) + (b.get("elements") or []) + (c.get("elements") or []))
            fe = np.nanmean([_prop(a, "formation_energy_per_atom"), _prop(b, "formation_energy_per_atom"), _prop(c, "formation_energy_per_atom")])
            rho = np.nanmean([_prop(a, "density"), _prop(b, "density"), _prop(c, "density")])
            eh = np.nanmean([_prop(a, "energy_above_hull"), _prop(b, "energy_above_hull"), _prop(c, "energy_above_hull")])
            bg = np.nanmean([_prop(a, "band_gap"), _prop(b, "band_gap"), _prop(c, "band_gap")])
            w1, w2, w3 = weights["w1"], weights["w2"], weights["w3"]
            s = (w1 * -fe) + (w2 * -rho) + (w3 * -eh) + (bg * 0.05) - _chem_penalty(elems)
            combos.append({
                "type": "ternary",
                "members": [a["formula"], b["formula"], c["formula"]],
                "props": {"formation_energy_per_atom": fe, "density": rho, "energy_above_hull": eh, "band_gap": bg},
                "synergy_score": float(s),
            })

        combos_sorted = sorted(combos, key=lambda x: x["synergy_score"], reverse=True)
        payload = {
            "agent": "MaterialsCombinerAgent",
            "status": "ok",
            "weights": weights,
            "num_combinations": len(combos_sorted),
            "top_combinations": combos_sorted[:12],
            "timestamp": _utcnow()
        }
        _log(f"Combiner: {len(combos_sorted)} combos → top {len(payload['top_combinations'])}")
        return json.dumps(payload, ensure_ascii=False)

# -----------------------------------------------------------------------------
# 3) SwarmPlannerAgent — multi-drone cooperative planning
# -----------------------------------------------------------------------------
class SwarmPlannerAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        rng = np.random.default_rng()
        n = int(rng.integers(3, 10))
        area = rng.choice(["north_sector", "south_sector", "ridge_zone", "valley_edge"])
        plan = [{"drone_id": f"UAV-{i+1}", "sector": area, "altitude_m": round(float(rng.uniform(80, 150)), 1)}
                for i in range(n)]
        payload = {"agent": "SwarmPlannerAgent", "status": "ok", "num_drones": n, "assignments": plan, "timestamp": _utcnow()}
        _log(f"Swarm plan created for {n} drones in {area}")
        return json.dumps(payload, ensure_ascii=False)

# -----------------------------------------------------------------------------
# 4) SimAgent — containment efficiency simulation
# -----------------------------------------------------------------------------
class SimAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        rng = np.random.default_rng()
        baseline = round(float(rng.uniform(1000, 5000)), 1)
        mitigated = round(baseline * float(rng.uniform(0.3, 0.8)), 1)
        eff = round((baseline - mitigated) / baseline, 3)
        payload = {"agent": "SimAgent", "status": "ok", "baseline_area_ha": baseline, "mitigated_area_ha": mitigated, "efficiency": eff, "timestamp": _utcnow()}
        _log(f"Simulation: efficiency={eff}, saved={baseline - mitigated:.1f} ha")
        return json.dumps(payload, ensure_ascii=False)

# -----------------------------------------------------------------------------
# 5) ActuationAgent — mission dispatch
# -----------------------------------------------------------------------------
class ActuationAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        payload = {"agent": "ActuationAgent", "status": "executed", "timestamp": _utcnow()}
        _log("Actuation dispatched")
        return json.dumps(payload, ensure_ascii=False)

# -----------------------------------------------------------------------------
# 6) CommAgent — summaries
# -----------------------------------------------------------------------------
class CommAgent(TextAgent):
    def handle(self, text: str = "") -> str:
        js = _safe_json(text)
        agent = js.get("agent")
        if agent == "MaterialsAgent":
            return f"[Materials] {len(js.get('base_candidates', []))} materials ({js.get('status')})"
        if agent == "MaterialsCombinerAgent":
            return f"[Combiner] {js.get('num_combinations')} combos"
        if agent == "SimAgent":
            return f"[Simulation] eff={js.get('efficiency')} saved {js.get('baseline_area_ha')}→{js.get('mitigated_area_ha')} ha"
        if agent == "SwarmPlannerAgent":
            sector = (js.get('assignments') or [{}])[0].get('sector', 'unknown')
            return f"[Swarm] {js.get('num_drones')} drones in {sector}"
        if agent == "ActuationAgent":
            return "[Actuation] mission executed successfully"
        return "[Comm] unrecognized payload"

# -----------------------------------------------------------------------------
# Instantiate agents
# -----------------------------------------------------------------------------
materials_context_builder = MaterialsContextBuilder()
materials_agent           = MaterialsAgent()
materials_combiner_agent  = MaterialsCombinerAgent()
swarm_planner_agent       = SwarmPlannerAgent()
sim_agent                 = SimAgent()
actuation_agent           = ActuationAgent()
comm_agent                = CommAgent()

_log("Framework B agents initialized: MaterialsContextBuilder, MaterialsAgent, MaterialsCombinerAgent, SwarmPlannerAgent, SimAgent, ActuationAgent, CommAgent")
print("Framework B ready — operational layer initialized successfully.")

# -----------------------------------------------------------------------------
# Core entrypoint — run_material_framework
# -----------------------------------------------------------------------------
def run_material_framework(summary_A_text: Union[str, dict], origin: str = "manual") -> str:
    try:
        js = _safe_json(summary_A_text)
        region = js.get("region", "unknown")
        slope  = _get_signal(js, "slope_mean_deg", 10.0)

        ctx  = json.loads(materials_context_builder.handle(js))
        mats = json.loads(materials_agent.handle(ctx))
        comb = json.loads(materials_combiner_agent.handle(mats))
        plan = json.loads(swarm_planner_agent.handle(comb))
        sim  = json.loads(sim_agent.handle(plan))
        act  = json.loads(actuation_agent.handle(sim))

        summary_text = " | ".join([
            comm_agent.handle(mats),
            comm_agent.handle(comb),
            comm_agent.handle(plan),
            comm_agent.handle(sim),
            comm_agent.handle(act),
        ])

        payload = {
            "agent": "MaterialsFramework",
            "status": "ok",
            "origin": origin,
            "wildfire_context": {"region": region, "slope_mean_deg": slope},
            "weights": ctx.get("context", {}).get("weights"),
            "num_base_candidates": len(mats.get("base_candidates") or []),
            "num_combinations": comb.get("num_combinations"),
            "top_combinations": comb.get("top_combinations")[:6],
            "summary_text": summary_text,
            "summary_hash": hashlib.md5(json.dumps(comb.get("top_combinations")[:6], ensure_ascii=False).encode("utf-8")).hexdigest()[:10],
            "timestamp": _utcnow()
        }

        text = "[Framework B] " + json.dumps(payload, ensure_ascii=False)
        print(f"Framework B executed successfully for region={region}")
        return text

    except Exception as e:
        _log(f"Framework B error: {type(e).__name__} → {e}", origin)
        traceback.print_exc()
        return json.dumps({"agent": "MaterialsFramework", "status": "error", "error_type": type(e).__name__, "message": str(e), "timestamp": _utcnow()})


[2025-11-04T18:37:32+00:00][FrameworkB] Framework B agents initialized: MaterialsContextBuilder, MaterialsAgent, MaterialsCombinerAgent, SwarmPlannerAgent, SimAgent, ActuationAgent, CommAgent
Framework B ready — operational layer initialized successfully.


# -------------------------------------------------------------------------
## ==== Cell 4.5 — AG² System Test & Validation ====

Executes a **complete end-to-end validation** of the AG² system to ensure all components  
(**Framework A → Coordinator → Framework B**) are operational, synchronized, and producing consistent outputs.

It performs:
1. Full pipeline execution using the current environment (`WF_REGION`, `WF_DATE_FROM`, `WF_DATE_TO`).
2. Validation of structured outputs (`summary_A`, `summary_B`) via Pydantic models.
3. Generation of key performance metrics (fire counts, slope mean, materials suggested, efficiency, runtime).
4. Tail preview of Coordinator and Framework B logs for rapid debugging.

All validation artifacts are automatically versioned and stored under `/reports/history/`.
# -------------------------------------------------------------------------

In [14]:
# ==== WildfiresAI — Cell 4.5: AG² System Test & Validation ====

from __future__ import annotations
import os, json, datetime as dt
from pathlib import Path
import pandas as pd

# ---------------------------------------------------------------------------
# 1) Environment setup
# ---------------------------------------------------------------------------
PROJECT_ROOT = globals().get("PROJECT_ROOT", Path.cwd())
REPORTS_DIR  = PROJECT_ROOT / "reports"
HISTORY_DIR  = REPORTS_DIR / "history"
LOG_DIR      = REPORTS_DIR / "logs"

for d in (REPORTS_DIR, HISTORY_DIR, LOG_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("🔍 AG² System Validation initialized.")
print(f" Using environment region={os.getenv('WF_REGION', 'GLOBAL')}")

# ---------------------------------------------------------------------------
# 2) Execute full A→B pipeline
# ---------------------------------------------------------------------------
try:
    start = dt.datetime.now(dt.UTC)
    summary_A, summary_B = coordinator.pipeline(
        region=os.getenv("WF_REGION", "GLOBAL"),
        date_from=os.getenv("WF_DATE_FROM", "unknown_from"),
        date_to=os.getenv("WF_DATE_TO", "unknown_to"),
    )
    duration = (dt.datetime.now(dt.UTC) - start).total_seconds()
    print(f"\n AG² pipeline executed successfully in {duration:.2f} seconds.")
except Exception as e:
    print(f" AG² pipeline failed: {e}")
    raise

# ---------------------------------------------------------------------------
# 3) Validate structured outputs
# ---------------------------------------------------------------------------
def validate_summary(summary_obj: dict, expected_keys: list[str], name: str):
    """Check presence of expected keys in the summary object."""
    missing = [k for k in expected_keys if k not in summary_obj]
    if missing:
        print(f"  Missing fields in {name}: {missing}")
    else:
        print(f"  {name} structure OK ({len(summary_obj)} keys).")

A_dict = summary_A.model_dump()
B_dict = summary_B.model_dump()

validate_summary(A_dict, ["agent", "region", "window", "signals"], "Summary A")
validate_summary(B_dict, ["agent", "status", "candidates_top3"], "Summary B")

# ---------------------------------------------------------------------------
# 4) Generate validation report
# ---------------------------------------------------------------------------
fires = (A_dict.get("counts") or {}).get("fires", "?")
slope = (A_dict.get("signals") or {}).get("slope_mean_deg", "?")
mats  = len(B_dict.get("candidates_top3") or [])
eff   = (B_dict.get("wildfire_context") or {}).get("efficiency", "?")

summary_report = {
    "run_id": coordinator.run_id,
    "region": A_dict.get("region", "?"),
    "fires_detected": fires,
    "slope_mean_deg": slope,
    "materials_suggested": mats,
    "efficiency": eff,
    "timestamp": dt.datetime.now(dt.UTC).isoformat(timespec="seconds"),
    "execution_seconds": duration,
}

report_path = HISTORY_DIR / f"{coordinator.run_id}_validation_summary.json"
with open(report_path, "w", encoding="utf-8") as f:
    json.dump(summary_report, f, ensure_ascii=False, indent=2)

print(f"\n🧾 Validation summary written to: {report_path}")

# ---------------------------------------------------------------------------
# 5) Display condensed results
# ---------------------------------------------------------------------------
print("\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(" AG² Validation Summary")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f" Region: {summary_report['region']}")
print(f" Fires detected: {summary_report['fires_detected']}")
print(f" Mean slope: {summary_report['slope_mean_deg']}°")
print(f" Materials suggested: {summary_report['materials_suggested']}")
print(f" Containment efficiency: {summary_report['efficiency']}")
print(f" Runtime: {summary_report['execution_seconds']:.2f}s")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

# ---------------------------------------------------------------------------
# 6) Log tail preview
# ---------------------------------------------------------------------------
def tail_log(path: Path, n: int = 10):
    """Show last N lines of a log file (if exists)."""
    if path.exists():
        lines = path.read_text(encoding="utf-8").splitlines()[-n:]
        print(f"\n Last {n} lines from {path.name}:")
        for line in lines:
            print(" ", line)
    else:
        print(f"(No log found at {path})")

tail_log(LOG_DIR / "coordinator.log")
tail_log(LOG_DIR / "framework_b.log")

print("\n AG² System Validation completed successfully.")


🔍 AG² System Validation initialized.
 Using environment region=EUROPE
[2025-11-04T19:11:45+00:00][FrameworkB] ══════════ AG² PIPELINE START ══════════
[2025-11-04T19:11:45+00:00][FrameworkB] Running Framework A (Wildfire Intelligence)…
[2025-11-04T19:11:45+00:00][FrameworkB] Pipeline failed: run_wildfire_framework() got an unexpected keyword argument 'region'
[2025-11-04T19:11:45+00:00][FrameworkB] Traceback (most recent call last):
  File "/var/folders/wn/mpnj0v3s1_1fd88s9l4c9pth0000gn/T/ipykernel_1114/2100843501.py", line 214, in pipeline
    A = self.run_A(**kwargs)
  File "/var/folders/wn/mpnj0v3s1_1fd88s9l4c9pth0000gn/T/ipykernel_1114/2100843501.py", line 137, in run_A
    text = globals()["run_wildfire_framework"](**kwargs)
TypeError: run_wildfire_framework() got an unexpected keyword argument 'region'
[2025-11-04T19:11:45+00:00][FrameworkB] ══════════ AG² PIPELINE END ══════════
 AG² pipeline failed: run_wildfire_framework() got an unexpected keyword argument 'region'


TypeError: run_wildfire_framework() got an unexpected keyword argument 'region'

# -------------------------------------------------------------------------
## ==== Cell 5 — AG² Interactive Command Console ====

Implements the interactive human–AI interface for the AG² System.

It bridges human operators with the dual frameworks:
- **Framework A (Wildfire Intelligence)** — environmental and risk analytics.  
- **Framework B (Materials Intelligence)** — material response and containment planning.  
- **LLM Orchestrator (Cell 4.3)** — interprets natural-language intent and activates the appropriate framework(s).

### Operational Flow
Human → Query → LLM Assistant → Orchestrator → Coordinator → Framework A → Framework B → Report

### Capabilities
- Context-aware interpretation via GPT-5.  
- Automatic region and timeframe detection.  
- Persistent audit logging in `/reports/chat_log.json`.  
- Fully functional for field or command-center deployment.

All interactions, logs, and reports are versioned and traceable.

# -------------------------------------------------------------------------




In [ ]:
# ==== Cell 5 — AG² Interactive Command Console (LLM + Map, Field-Ready) ====

from __future__ import annotations
import os, json, datetime as dt
from pathlib import Path
from typing import Optional, Dict, Any, Tuple
import pandas as pd

# GIS (map rendering)
import folium
from folium.plugins import HeatMap

# ---------------------------------------------------------------------------
# 1) Environment & directories
# ---------------------------------------------------------------------------
PROJECT_ROOT = globals().get("PROJECT_ROOT", Path.cwd())
REPORTS_DIR  = PROJECT_ROOT / "reports"
MAPS_DIR     = REPORTS_DIR / "maps"
CHAT_LOG     = REPORTS_DIR / "chat_log.json"
for d in (REPORTS_DIR, MAPS_DIR):
    d.mkdir(parents=True, exist_ok=True)

# These should already be defined in previous cells (4.1/4.2), but default safely:
WF_REGION     = globals().get("WF_REGION", os.getenv("WF_REGION", "GLOBAL")).upper()
WF_DATE_FROM  = globals().get("WF_DATE_FROM", os.getenv("WF_DATE_FROM", "unknown_from"))
WF_DATE_TO    = globals().get("WF_DATE_TO", os.getenv("WF_DATE_TO", "unknown_to"))
PROCESSED_DIR = (PROJECT_ROOT / "data" / "processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

USE_LLM = bool(os.getenv("OPENAI_API_KEY"))

# ---------------------------------------------------------------------------
# 2) Region / timeframe detection (fallback heuristics if LLM is off)
# ---------------------------------------------------------------------------
def detect_region(question: str) -> str:
    """Heuristic region extractor for offline mode."""
    q = question.lower()
    mapping = {
        "spain": "SPAIN", "españa": "SPAIN", "portugal": "PORTUGAL",
        "france": "FRANCE", "italy": "ITALY", "usa": "USA",
        "united states": "USA", "canada": "CANADA",
        "australia": "AUSTRALIA", "europe": "EUROPE",
        "africa": "AFRICA", "asia": "ASIA", "global": "GLOBAL"
    }
    for k, v in mapping.items():
        if k in q:
            return v
    return "GLOBAL"

def detect_days(question: str) -> int:
    """Approximate time window in days from natural language cues."""
    q = question.lower()
    if "today" in q or "hoy" in q: return 1
    if "yesterday" in q or "ayer" in q: return 2
    if "week" in q or "semana" in q: return 7
    if "month" in q or "mes" in q: return 30
    if "year" in q or "año" in q: return 365
    return 7

# ---------------------------------------------------------------------------
# 3) LLM assistant (uses GPT-5 when available)
# ---------------------------------------------------------------------------
def llm_refine_query(question: str) -> Dict[str, Any]:
    """
    Use GPT-5 to convert user's natural language question into a structured command.
    Returns JSON: {intent, region, period_days, parameters?}
    """
    if not USE_LLM:
        return {
            "intent": "analyze_wildfires",
            "region": detect_region(question),
            "period_days": detect_days(question),
            "context": "heuristic",
            "confidence": 0.6,
        }

    try:
        from openai import OpenAI
        client = OpenAI()
        system_prompt = (
            "You are AG² Command LLM for WildfiresAI. "
            "Given a human query, output ONLY JSON with keys: "
            "{intent, region, period_days, parameters}. "
            "intent∈{analyze_wildfires, materials_query, both}. "
            "Return valid JSON only."
        )
        user_prompt = f"Query: {question}"

        resp = client.chat.completions.create(
            model="gpt-5",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.2,
            max_tokens=300,
        )
        text = resp.choices[0].message.content.strip()
        parsed = json.loads(text)
        parsed["context"] = "llm"
        parsed["confidence"] = 0.95
        return parsed
    except Exception as e:
        print(f"LLM failed ({e}); using heuristic fallback.")
        return {
            "intent": "analyze_wildfires",
            "region": detect_region(question),
            "period_days": detect_days(question),
            "context": "fallback",
            "confidence": 0.5,
        }

# ---------------------------------------------------------------------------
# 4) Data loading + map rendering (robust to schema variants)
# ---------------------------------------------------------------------------
def _find_processed_file(region: str) -> Optional[Path]:
    """
    Locate a plausible processed parquet: strict name first, then newest match.
    """
    strict = PROCESSED_DIR / f"fires_terrain_{region}_{WF_DATE_FROM}_{WF_DATE_TO}.parquet"
    if strict.exists():
        return strict

    candidates = sorted(
        PROCESSED_DIR.glob(f"fires_terrain_{region}*.parquet"),
        key=lambda p: p.stat().st_mtime,
        reverse=True,
    )
    return candidates[0] if candidates else None

def _normalize_latlon(df: pd.DataFrame) -> Optional[pd.DataFrame]:
    """
    Normalize lat/lon columns to ['lat','lon'] and drop NA rows for mapping.
    Accepts columns {lat/lon} or {latitude/longitude}.
    """
    cols = {c.lower(): c for c in df.columns}
    lat = cols.get("lat") or cols.get("latitude")
    lon = cols.get("lon") or cols.get("longitude")
    if not lat or not lon:
        return None
    out = df.rename(columns={lat: "lat", lon: "lon"})
    out = out[["lat", "lon"]].dropna()
    return out

def render_heatmap_from_processed(region: str) -> Optional[Path]:
    """
    Build a folium heatmap from the latest processed parquet. Returns HTML path.
    If no data is available, returns None.
    """
    path = _find_processed_file(region)
    if not path or not path.exists():
        print("Map: no processed parquet found.")
        return None

    try:
        df = pd.read_parquet(path)
        points = _normalize_latlon(df)
        if points is None or points.empty:
            print("Map: no suitable lat/lon columns for rendering.")
            return None

        m = folium.Map(location=[points["lat"].mean(), points["lon"].mean()], zoom_start=5, control_scale=True)
        HeatMap(points[["lat", "lon"]].values.tolist(), radius=9, blur=15, max_zoom=8).add_to(m)

        ts = dt.datetime.now(dt.UTC).strftime("%Y%m%d_%H%M%S")
        out_html = MAPS_DIR / f"ag2_map_{region}_{ts}.html"
        m.save(out_html)
        print(f"Map saved to {out_html}")
        return out_html
    except Exception as e:
        print(f"Map rendering failed: {e}")
        return None

# ---------------------------------------------------------------------------
# 5) Human-readable summarizer for A→B results
# ---------------------------------------------------------------------------
def summarize_agents(A: dict, B: dict) -> str:
    slope = (A.get("signals") or {}).get("slope_mean_deg", "?")
    fires = (A.get("counts")  or {}).get("fires", "?")
    mats  = len(B.get("candidates_top3", []))
    # Efficiency is optional; only include if present in B
    eff = (B.get("wildfire_context") or {}).get("efficiency")
    base = f"[AG2] Region={A.get('region','?')} | Fires={fires} | Slope≈{slope}° | Materials={mats}"
    return base if eff is None else f"{base} | Efficiency={eff}"

# ---------------------------------------------------------------------------
# 6) Interactive loop — LLM-enhanced + map
# ---------------------------------------------------------------------------
def ag2_chat():
    """Interactive console with LLM orchestration and map output."""
    print("AG2 Interactive Console (LLM + Map) ready.")
    print("Type your question (e.g., 'Fires in Spain last month') or 'exit' to quit.")

    chat_history = []
    while True:
        q = input("\nAsk WildfiresAI: ").strip()
        if q.lower() in {"exit", "quit"}:
            print("Session ended.")
            break

        # 6.1 Interpret query with LLM (or heuristics)
        print("Interpreting intent via LLM Orchestrator...")
        intent_info = llm_refine_query(q)
        region = (intent_info.get("region") or WF_REGION).upper()
        period = int(intent_info.get("period_days", 7))
        intent = intent_info.get("intent", "analyze_wildfires")
        print(f"→ Intent: {intent} | Region: {region} | Period: {period} days")

        # 6.2 Compute window from period_days
        end_dt = dt.datetime.now(dt.UTC).date()
        start_dt = end_dt - dt.timedelta(days=period)
        date_from = start_dt.isoformat()
        date_to   = end_dt.isoformat()

        # 6.3 Run AG² pipeline end-to-end (Coordinator → A → B)
        print("Running AG2 pipeline...")
        t0 = dt.datetime.now(dt.UTC)
        try:
            A, B = coordinator.pipeline(region=region, date_from=date_from, date_to=date_to)
            summary = summarize_agents(A.model_dump(), B.model_dump())
            print(f"\n{summary}")

            # 6.4 Render map (best effort)
            map_path = render_heatmap_from_processed(region)

            # 6.5 Build response object
            record = {
                "question": q,
                "intent": intent,
                "region": region,
                "period_days": period,
                "llm_context": intent_info.get("context", "none"),
                "confidence": intent_info.get("confidence", 0),
                "answer": summary,
                "map_html": str(map_path) if map_path else None,
                "date_from": date_from,
                "date_to": date_to,
                "timestamp": dt.datetime.now(dt.UTC).isoformat(timespec="seconds"),
            }
            chat_history.append(record)

            if map_path:
                print("If the map does not render, trust this Notebook: File → Trust Notebook")

        except Exception as e:
            print(f"Pipeline error: {e}")
            chat_history.append({
                "question": q,
                "error": str(e),
                "timestamp": dt.datetime.now(dt.UTC).isoformat(timespec="seconds"),
            })

        # 6.6 Timing
        t1 = dt.datetime.now(dt.UTC)
        print(f"Completed in {(t1 - t0).total_seconds():.2f}s.")
        print("--------------------------------------------------")

    # 6.7 Persist full conversation
    with open(CHAT_LOG, "w", encoding="utf-8") as f:
        json.dump(chat_history, f, ensure_ascii=False, indent=2)
    print(f"Chat history saved to {CHAT_LOG}")

# ---------------------------------------------------------------------------
# 7) Launch console
# ---------------------------------------------------------------------------
ag2_chat()
